# Sistema de Minería de Texto para Mejora de Contenido Ambiental

Este notebook implementa un sistema completo de procesamiento y mejora de texto utilizando técnicas de NLP y el modelo BERT. El sistema está diseñado para mejorar textos relacionados con concientización ambiental, optimizando títulos, descripciones y contenido para un panel de administración.

## Estructura del Sistema

El sistema sigue las siguientes fases de procesamiento:

1. **Ingesta del texto**: Entrada de texto desde diferentes fuentes
2. **Limpieza básica**: Eliminación de caracteres especiales, espacios, etc.
3. **Tokenización**: División del texto en tokens (palabras, frases)
4. **Normalización**: Conversión a minúsculas, eliminación de acentos, etc.
5. **Eliminación de ruido**: Eliminación de stopwords y contenido irrelevante
6. **Lematización/Stemming**: Reducción de palabras a su forma base
7. **Procesamiento con BERT**: Mejora del texto utilizando modelos de lenguaje avanzados
8. **Generación de salida**: Texto mejorado con enfoque personalizable

Cada módulo se implementa de forma independiente para facilitar su comprensión y posible reutilización.

## Instalación de Dependencias

Primero, instalemos las bibliotecas necesarias para el procesamiento de texto y el uso de BERT.

In [40]:
# Importación de bibliotecas con manejo de errores
import re
import unicodedata
import json
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import warnings

# Configuraciones generales
warnings.filterwarnings('ignore')

# Importar bibliotecas científicas básicas
try:
    import numpy as np
    import pandas as pd
    print(f"✓ NumPy {np.__version__} y Pandas {pd.__version__} cargados")
except ImportError as e:
    print(f"❌ Error con NumPy/Pandas: {e}")
    raise

# Importar bibliotecas de visualización
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.style.use('default')  # Usar estilo por defecto más compatible
    sns.set_palette("husl")
    print("✓ Matplotlib y Seaborn cargados")
except ImportError as e:
    print(f"❌ Error con visualización: {e}")

# Importar NLTK
try:
    import nltk
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import SnowballStemmer, WordNetLemmatizer
    print(f"✓ NLTK {nltk.__version__} cargado")
except ImportError as e:
    print(f"❌ Error con NLTK: {e}")
    raise

# Importar spaCy
try:
    import spacy
    print(f"✓ spaCy {spacy.__version__} cargado")
except ImportError as e:
    print(f"❌ Error con spaCy: {e}")
    raise

# Configurar PyTorch y Transformers con manejo de errores
TORCH_AVAILABLE = False
TRANSFORMERS_AVAILABLE = False

# Configurar variables de entorno para evitar conflictos con TensorFlow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['USE_TF'] = 'NO'
os.environ['USE_TORCH'] = 'YES'

try:
    import torch
    TORCH_AVAILABLE = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"✓ PyTorch {torch.__version__} cargado")
    print(f"✓ Dispositivo configurado: {device}")
except ImportError as e:
    print(f"⚠️ PyTorch no disponible: {e}")
    device = 'cpu'

try:
    from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
    from transformers import BertTokenizer, BertModel
    import transformers
    transformers.utils.logging.set_verbosity_error()
    TRANSFORMERS_AVAILABLE = True
    print(f"✓ Transformers {transformers.__version__} cargado (solo PyTorch)")
except ImportError as e:
    print(f"⚠️ Transformers no disponible: {e}")
    print("El sistema funcionará sin capacidades de BERT")

# Descargar recursos necesarios de NLTK
print("\nDescargando recursos de NLTK...")
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    try:
        nltk.download('punkt_tab', quiet=True)
    except:
        pass  # Este recurso puede no estar disponible en todas las versiones
    print("✓ Recursos de NLTK descargados")
except Exception as e:
    print(f"⚠️ Advertencia con recursos NLTK: {e}")

# Cargar modelo de spaCy para español
print("Cargando modelo de spaCy para español...")
try:
    nlp = spacy.load('es_core_news_sm')
    print("✓ Modelo de spaCy cargado exitosamente")
except OSError:
    print("Descargando modelo de spaCy para español...")
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_sm"])
        nlp = spacy.load('es_core_news_sm')
        print("✓ Modelo de spaCy descargado y cargado exitosamente")
    except Exception as e:
        print(f"❌ Error descargando modelo de spaCy: {e}")
        print("Usando modelo en blanco como fallback...")
        nlp = spacy.blank('es')

# Configurar stopwords en español
try:
    spanish_stopwords = set(stopwords.words('spanish'))
    
    # Añadir stopwords personalizadas para contexto ambiental
    environmental_stopwords = {
        'además', 'asimismo', 'también', 'igualmente', 'por tanto', 
        'sin embargo', 'no obstante', 'por consiguiente'
    }
    spanish_stopwords.update(environmental_stopwords)
    print("✓ Stopwords configuradas")
except Exception as e:
    print(f"⚠️ Error configurando stopwords: {e}")
    spanish_stopwords = set()

# Configurar stemmer y lemmatizer
try:
    stemmer = SnowballStemmer('spanish')
    lemmatizer = WordNetLemmatizer()
    print("✓ Stemmer y Lemmatizer configurados")
except Exception as e:
    print(f"⚠️ Error configurando stemmer/lemmatizer: {e}")

# Configurar modelos BERT (se cargarán cuando sea necesario)
BERT_MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-uncased' if TRANSFORMERS_AVAILABLE else None

# Resumen del estado del sistema
print("\n" + "=" * 50)
print("ESTADO DEL SISTEMA:")
print(f"✓ Bibliotecas básicas: Cargadas")
print(f"✓ NumPy/Pandas: Cargadas")
print(f"✓ NLTK: Cargado")
print(f"✓ spaCy: Cargado")
print(f"{'✓' if TORCH_AVAILABLE else '⚠️'} PyTorch: {'Disponible' if TORCH_AVAILABLE else 'No disponible'}")
print(f"{'✓' if TRANSFORMERS_AVAILABLE else '⚠️'} Transformers: {'Disponible' if TRANSFORMERS_AVAILABLE else 'No disponible'}")
print(f"{'✓' if TRANSFORMERS_AVAILABLE else '⚠️'} BERT: {'Disponible' if TRANSFORMERS_AVAILABLE else 'No disponible'}")
print(f"✓ Dispositivo: {device}")

if BERT_MODEL_NAME:
    print(f"✓ Modelo BERT configurado: {BERT_MODEL_NAME}")
else:
    print("⚠️ BERT no disponible - usando métodos alternativos")

print("=" * 50)
print("✓ CONFIGURACIÓN COMPLETADA")

# Variables globales para uso en el resto del código
SYSTEM_CONFIG = {
    'torch_available': TORCH_AVAILABLE,
    'transformers_available': TRANSFORMERS_AVAILABLE,
    'bert_model_name': BERT_MODEL_NAME,
    'device': device,
    'nlp_model': nlp,
    'spanish_stopwords': spanish_stopwords,
    'stemmer': stemmer if 'stemmer' in locals() else None,
    'lemmatizer': lemmatizer if 'lemmatizer' in locals() else None
}

print("✓ Variables del sistema configuradas")


✓ NumPy 1.24.4 y Pandas 2.0.3 cargados
✓ Matplotlib y Seaborn cargados
✓ NLTK 3.8.1 cargado
✓ spaCy 3.7.2 cargado
✓ PyTorch 2.7.1+cpu cargado
✓ Dispositivo configurado: cpu
✓ Transformers 4.53.2 cargado (solo PyTorch)

Descargando recursos de NLTK...
✓ Recursos de NLTK descargados
Cargando modelo de spaCy para español...
✓ Modelo de spaCy cargado exitosamente
✓ Stopwords configuradas
✓ Stemmer y Lemmatizer configurados

ESTADO DEL SISTEMA:
✓ Bibliotecas básicas: Cargadas
✓ NumPy/Pandas: Cargadas
✓ NLTK: Cargado
✓ spaCy: Cargado
✓ PyTorch: Disponible
✓ Transformers: Disponible
✓ BERT: Disponible
✓ Dispositivo: cpu
✓ Modelo BERT configurado: dccuchile/bert-base-spanish-wwm-uncased
✓ CONFIGURACIÓN COMPLETADA
✓ Variables del sistema configuradas


Modulo 1: Ingesta del Texto

In [41]:
# ===== MÓDULO 1: INGESTA DEL TEXTO =====

class TextIngestion:
    """
    Módulo para la ingesta de texto desde diferentes fuentes.
    Permite entrada manual, desde archivos o URLs.
    """
    
    def __init__(self):
        self.supported_formats = ['txt', 'csv', 'json']
    
    def ingest_manual_text(self, text):
        """
        Ingesta de texto manual directo
        """
        if not isinstance(text, str):
            raise ValueError("El texto debe ser una cadena de caracteres")
        
        return {
            'original_text': text,
            'source': 'manual',
            'length': len(text),
            'word_count': len(text.split())
        }
    
    def ingest_from_file(self, file_path):
        """
        Ingesta de texto desde archivo
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
            
            return {
                'original_text': text,
                'source': f'file: {file_path}',
                'length': len(text),
                'word_count': len(text.split())
            }
        except Exception as e:
            raise Exception(f"Error al leer archivo: {str(e)}")
    
    def validate_text(self, text_data):
        """
        Validación básica del texto ingresado
        """
        text = text_data['original_text']
        
        validations = {
            'is_empty': len(text.strip()) == 0,
            'min_length': len(text) >= 10,
            'has_letters': bool(re.search(r'[a-zA-ZáéíóúÁÉÍÓÚñÑ]', text)),
            'encoding_issues': '�' in text
        }
        
        return validations

# Ejemplo de uso del módulo de ingesta
print("=== MÓDULO 1: INGESTA DEL TEXTO ===")

# Crear instancia del módulo
ingestion = TextIngestion()

# Texto de ejemplo relacionado con medio ambiente
sample_text = """
El cambio climático representa uno de los mayores desafíos de nuestro tiempo. 
Cambio Climatico mayor desafio tiempo
La deforestación, la contaminación del aire y el uso excesivo de combustibles fósiles 
están afectando gravemente nuestro planeta. Es fundamental implementar energías renovables 
y promover prácticas sostenibles para proteger el medio ambiente para las futuras generaciones.
"""

# Ingestar el texto
text_data = ingestion.ingest_manual_text(sample_text)
print("Texto ingresado exitosamente:")
print(f"- Fuente: {text_data['source']}")
print(f"- Longitud: {text_data['length']} caracteres")
print(f"- Palabras: {text_data['word_count']} palabras")

# Validar el texto
validations = ingestion.validate_text(text_data)
print("\nValidaciones:")
for key, value in validations.items():
    print(f"- {key}: {value}")


=== MÓDULO 1: INGESTA DEL TEXTO ===
Texto ingresado exitosamente:
- Fuente: manual
- Longitud: 393 caracteres
- Palabras: 53 palabras

Validaciones:
- is_empty: False
- min_length: True
- has_letters: True
- encoding_issues: False


Modulo 2: Limpieza Basica

In [42]:
# ===== MÓDULO 2: LIMPIEZA BÁSICA =====

class TextCleaner:
    """
    Módulo para la limpieza básica del texto.
    Elimina caracteres especiales, espacios extra, URLs, emails, etc.
    """
    
    def __init__(self):
        # Patrones regex para diferentes tipos de limpieza
        self.url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        self.email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
        self.phone_pattern = re.compile(r'(\+?[0-9]{1,3}[-.\s]?)?(\(?[0-9]{3}\)?[-.\s]?[0-9]{3}[-.\s]?[0-9]{4})')
        self.html_pattern = re.compile(r'<[^>]+>')
        self.special_chars_pattern = re.compile(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúÁÉÍÓÚñÑüÜ]')
        self.multiple_spaces_pattern = re.compile(r'\s+')
        self.multiple_punctuation_pattern = re.compile(r'([.!?]){2,}')
    
    def remove_urls(self, text: str) -> str:
        """Elimina URLs del texto"""
        return self.url_pattern.sub('', text)
    
    def remove_emails(self, text: str) -> str:
        """Elimina direcciones de email del texto"""
        return self.email_pattern.sub('', text)
    
    def remove_phone_numbers(self, text: str) -> str:
        """Elimina números de teléfono del texto"""
        return self.phone_pattern.sub('', text)
    
    def remove_html_tags(self, text: str) -> str:
        """Elimina etiquetas HTML del texto"""
        return self.html_pattern.sub('', text)
    
    def remove_special_characters(self, text: str, keep_punctuation: bool = True) -> str:
        """
        Elimina caracteres especiales manteniendo letras, números y opcionalmente puntuación
        """
        if keep_punctuation:
            # Mantener puntuación básica y caracteres en español
            cleaned = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúÁÉÍÓÚñÑüÜ]', ' ', text)
        else:
            # Solo mantener letras, números y espacios
            cleaned = re.sub(r'[^\w\s áéíóúÁÉÍÓÚñÑüÜ]', ' ', text)
        
        return cleaned
    
    def normalize_whitespace(self, text: str) -> str:
        """Normaliza espacios en blanco múltiples"""
        # Reemplazar múltiples espacios con uno solo
        text = self.multiple_spaces_pattern.sub(' ', text)
        # Eliminar espacios al inicio y final
        return text.strip()
    
    def normalize_punctuation(self, text: str) -> str:
        """Normaliza puntuación repetida"""
        return self.multiple_punctuation_pattern.sub(r'\1', text)
    
    def remove_extra_newlines(self, text: str) -> str:
        """Elimina saltos de línea excesivos"""
        # Reemplazar múltiples saltos de línea con uno solo
        text = re.sub(r'\n+', '\n', text)
        # Reemplazar saltos de línea con espacios para texto continuo
        text = re.sub(r'\n', ' ', text)
        return text
    
    def clean_encoding_issues(self, text: str) -> str:
        """Corrige problemas de codificación comunes"""
        # Diccionario de reemplazos comunes
        encoding_fixes = {
            'Ã¡': 'á', 'Ã©': 'é', 'Ã­': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
            'Ã±': 'ñ', 'Ã¼': 'ü', 'Â': '', 'â€™': "'", 'â€œ': '"',
            'â€': '"', 'â€¦': '...', 'â€"': '-', '�': ''
        }
        
        for wrong, correct in encoding_fixes.items():
            text = text.replace(wrong, correct)
        
        return text
    
    def basic_clean(self, text: str, options: Dict = None) -> Dict:
        """
        Realiza limpieza básica completa del texto
        
        Args:
            text: Texto a limpiar
            options: Diccionario con opciones de limpieza
        
        Returns:
            Diccionario con texto limpio y estadísticas
        """
        if options is None:
            options = {
                'remove_urls': True,
                'remove_emails': True,
                'remove_phones': True,
                'remove_html': True,
                'remove_special_chars': True,
                'keep_punctuation': True,
                'normalize_whitespace': True,
                'normalize_punctuation': True,
                'remove_newlines': True,
                'fix_encoding': True
            }
        
        original_text = text
        original_length = len(text)
        
        # Aplicar limpiezas según opciones
        if options.get('fix_encoding', True):
            text = self.clean_encoding_issues(text)
        
        if options.get('remove_html', True):
            text = self.remove_html_tags(text)
        
        if options.get('remove_urls', True):
            text = self.remove_urls(text)
        
        if options.get('remove_emails', True):
            text = self.remove_emails(text)
        
        if options.get('remove_phones', True):
            text = self.remove_phone_numbers(text)
        
        if options.get('remove_newlines', True):
            text = self.remove_extra_newlines(text)
        
        if options.get('remove_special_chars', True):
            text = self.remove_special_characters(text, options.get('keep_punctuation', True))
        
        if options.get('normalize_punctuation', True):
            text = self.normalize_punctuation(text)
        
        if options.get('normalize_whitespace', True):
            text = self.normalize_whitespace(text)
        
        # Calcular estadísticas
        cleaned_length = len(text)
        reduction_percentage = ((original_length - cleaned_length) / original_length) * 100 if original_length > 0 else 0
        
        return {
            'original_text': original_text,
            'cleaned_text': text,
            'original_length': original_length,
            'cleaned_length': cleaned_length,
            'reduction_percentage': round(reduction_percentage, 2),
            'cleaning_options': options
        }

# Ejemplo de uso del módulo de limpieza
print("=== MÓDULO 2: LIMPIEZA BÁSICA ===")

# Crear instancia del limpiador
cleaner = TextCleaner()

# Texto de ejemplo con problemas comunes
dirty_text = """
¡¡¡El cambio climático!!! es un problema muy serio...    

Visita https://ejemplo.com para más información.
Contacto: info@medioambiente.org o llama al +1-555-123-4567

<p>La <strong>deforestación</strong> afecta nuestro planeta.</p>

Ã¡reas protegidas son fundamentales!!!   Â¿Qué podemos hacer???


"""

print("Texto original:")
print(repr(dirty_text))
print("\n" + "="*50)

# Limpiar el texto
result = cleaner.basic_clean(dirty_text)

print("Texto limpio:")
print(f"'{result['cleaned_text']}'")
print(f"\nEstadísticas de limpieza:")
print(f"- Longitud original: {result['original_length']} caracteres")
print(f"- Longitud limpia: {result['cleaned_length']} caracteres")
print(f"- Reducción: {result['reduction_percentage']}%")

# Ejemplo con opciones personalizadas
print("\n" + "="*30)
print("Limpieza con opciones personalizadas:")

custom_options = {
    'remove_urls': True,
    'remove_emails': False,  # Mantener emails
    'remove_phones': True,
    'remove_html': True,
    'remove_special_chars': True,
    'keep_punctuation': False,  # Eliminar toda la puntuación
    'normalize_whitespace': True,
    'normalize_punctuation': True,
    'remove_newlines': True,
    'fix_encoding': True
}

custom_result = cleaner.basic_clean(dirty_text, custom_options)
print(f"Texto con limpieza personalizada:")
print(f"'{custom_result['cleaned_text']}'")


=== MÓDULO 2: LIMPIEZA BÁSICA ===
Texto original:
'\n¡¡¡El cambio climático!!! es un problema muy serio...    \n\nVisita https://ejemplo.com para más información.\nContacto: info@medioambiente.org o llama al +1-555-123-4567\n\n<p>La <strong>deforestación</strong> afecta nuestro planeta.</p>\n\nÃ¡reas protegidas son fundamentales!!!   Â¿Qué podemos hacer???\n\n\n'

Texto limpio:
'El cambio climático! es un problema muy serio. Visita para más información. Contacto: o llama al La deforestación afecta nuestro planeta. áreas protegidas son fundamentales! Qué podemos hacer?'

Estadísticas de limpieza:
- Longitud original: 302 caracteres
- Longitud limpia: 192 caracteres
- Reducción: 36.42%

Limpieza con opciones personalizadas:
Texto con limpieza personalizada:
'El cambio climático es un problema muy serio Visita para más información Contacto info medioambiente org o llama al La deforestación afecta nuestro planeta áreas protegidas son fundamentales Qué podemos hacer'


Modulo 3: Tokenizacion

In [43]:
# ===== MÓDULO 3: TOKENIZACIÓN =====

class TextTokenizer:
    """
    Módulo para la tokenización del texto.
    Divide el texto en tokens (palabras, oraciones, párrafos) usando diferentes estrategias.
    """
    
    def __init__(self):
        # Configurar tokenizadores
        self.nlp = nlp  # spaCy model cargado anteriormente
        
        # Patrones para tokenización personalizada
        self.sentence_endings = re.compile(r'[.!?]+')
        self.word_pattern = re.compile(r'\b\w+\b')
        self.punctuation_pattern = re.compile(r'[^\w\s]')
        
    def tokenize_sentences(self, text: str, method: str = 'nltk') -> List[str]:
        """
        Tokeniza el texto en oraciones
        
        Args:
            text: Texto a tokenizar
            method: Método a usar ('nltk', 'spacy', 'regex')
        
        Returns:
            Lista de oraciones
        """
        if method == 'nltk':
            sentences = sent_tokenize(text, language='spanish')
        elif method == 'spacy':
            doc = self.nlp(text)
            sentences = [sent.text.strip() for sent in doc.sents]
        elif method == 'regex':
            # Método simple con regex
            sentences = self.sentence_endings.split(text)
            sentences = [s.strip() for s in sentences if s.strip()]
        else:
            raise ValueError("Método debe ser 'nltk', 'spacy' o 'regex'")
        
        return [s for s in sentences if len(s.strip()) > 0]
    
    def tokenize_words(self, text: str, method: str = 'nltk') -> List[str]:
        """
        Tokeniza el texto en palabras
        
        Args:
            text: Texto a tokenizar
            method: Método a usar ('nltk', 'spacy', 'regex')
        
        Returns:
            Lista de palabras
        """
        if method == 'nltk':
            words = word_tokenize(text, language='spanish')
        elif method == 'spacy':
            doc = self.nlp(text)
            words = [token.text for token in doc if not token.is_space]
        elif method == 'regex':
            words = self.word_pattern.findall(text)
        else:
            raise ValueError("Método debe ser 'nltk', 'spacy' o 'regex'")
        
        return words
    
    def tokenize_paragraphs(self, text: str) -> List[str]:
        """
        Tokeniza el texto en párrafos
        """
        # Dividir por dobles saltos de línea o líneas vacías
        paragraphs = re.split(r'\n\s*\n', text)
        return [p.strip() for p in paragraphs if p.strip()]
    
    def advanced_tokenization(self, text: str) -> Dict:
        """
        Tokenización avanzada usando spaCy con información lingüística
        
        Returns:
            Diccionario con tokens y sus propiedades
        """
        doc = self.nlp(text)
        
        tokens_info = []
        for token in doc:
            if not token.is_space:
                token_info = {
                    'text': token.text,
                    'lemma': token.lemma_,
                    'pos': token.pos_,  # Part of speech
                    'tag': token.tag_,  # Detailed POS tag
                    'is_alpha': token.is_alpha,
                    'is_stop': token.is_stop,
                    'is_punct': token.is_punct,
                    'is_digit': token.is_digit,
                    'shape': token.shape_,  # Forma de la palabra (Xxxx, dddd, etc.)
                    'is_title': token.is_title,
                    'is_lower': token.is_lower,
                    'is_upper': token.is_upper
                }
                tokens_info.append(token_info)
        
        return {
            'tokens': tokens_info,
            'total_tokens': len(tokens_info),
            'sentences': [sent.text for sent in doc.sents],
            'entities': [(ent.text, ent.label_) for ent in doc.ents]
        }
    
    def tokenize_by_pos(self, text: str) -> Dict[str, List[str]]:
        """
        Tokeniza y agrupa por categorías gramaticales
        
        Returns:
            Diccionario con tokens agrupados por POS
        """
        doc = self.nlp(text)
        
        pos_groups = {
            'sustantivos': [],
            'verbos': [],
            'adjetivos': [],
            'adverbios': [],
            'preposiciones': [],
            'conjunciones': [],
            'determinantes': [],
            'pronombres': [],
            'otros': []
        }
        
        pos_mapping = {
            'NOUN': 'sustantivos',
            'VERB': 'verbos',
            'ADJ': 'adjetivos',
            'ADV': 'adverbios',
            'ADP': 'preposiciones',
            'CONJ': 'conjunciones',
            'CCONJ': 'conjunciones',
            'DET': 'determinantes',
            'PRON': 'pronombres'
        }
        
        for token in doc:
            if token.is_alpha and not token.is_stop:
                category = pos_mapping.get(token.pos_, 'otros')
                pos_groups[category].append(token.lemma_.lower())
        
        # Eliminar duplicados manteniendo orden
        for category in pos_groups:
            pos_groups[category] = list(dict.fromkeys(pos_groups[category]))
        
        return pos_groups
    
    def extract_environmental_terms(self, text: str) -> Dict[str, List[str]]:
        """
        Extrae términos específicos relacionados con medio ambiente
        """
        doc = self.nlp(text)
        
        # Términos ambientales por categoría
        environmental_categories = {
            'problemas_ambientales': [],
            'soluciones': [],
            'recursos_naturales': [],
            'energia': [],
            'contaminacion': [],
            'conservacion': []
        }
        
        # Diccionarios de términos ambientales
        environmental_terms = {
            'problemas_ambientales': [
                'cambio climático', 'calentamiento global', 'deforestación', 
                'contaminación', 'extinción', 'desertificación', 'erosión'
            ],
            'soluciones': [
                'reciclaje', 'energía renovable', 'sostenible', 'sustentable',
                'conservación', 'reforestación', 'eficiencia energética'
            ],
            'recursos_naturales': [
                'agua', 'bosque', 'océano', 'biodiversidad', 'ecosistema',
                'fauna', 'flora', 'selva', 'río', 'lago'
            ],
            'energia': [
                'solar', 'eólica', 'hidroeléctrica', 'geotérmica', 
                'biomasa', 'combustible fósil', 'petróleo', 'carbón'
            ],
            'contaminacion': [
                'emisiones', 'gases', 'residuos', 'basura', 'tóxicos',
                'plástico', 'químicos', 'desechos'
            ],
            'conservacion': [
                'protección', 'preservación', 'área protegida', 'parque nacional',
                'reserva', 'santuario', 'hábitat'
            ]
        }
        
        text_lower = text.lower()
        
        for category, terms in environmental_terms.items():
            for term in terms:
                if term in text_lower:
                    environmental_categories[category].append(term)
        
        return environmental_categories
    
    def tokenization_statistics(self, text: str) -> Dict:
        """
        Genera estadísticas completas de tokenización
        """
        sentences = self.tokenize_sentences(text, 'spacy')
        words = self.tokenize_words(text, 'spacy')
        paragraphs = self.tokenize_paragraphs(text)
        advanced_tokens = self.advanced_tokenization(text)
        pos_groups = self.tokenize_by_pos(text)
        env_terms = self.extract_environmental_terms(text)
        
        # Calcular estadísticas
        word_lengths = [len(word) for word in words if word.isalpha()]
        sentence_lengths = [len(sent.split()) for sent in sentences]
        
        stats = {
            'total_characters': len(text),
            'total_words': len(words),
            'total_sentences': len(sentences),
            'total_paragraphs': len(paragraphs),
            'unique_words': len(set(word.lower() for word in words if word.isalpha())),
            'avg_word_length': np.mean(word_lengths) if word_lengths else 0,
            'avg_sentence_length': np.mean(sentence_lengths) if sentence_lengths else 0,
            'pos_distribution': {k: len(v) for k, v in pos_groups.items()},
            'environmental_terms_count': sum(len(terms) for terms in env_terms.values()),
            'entities_found': len(advanced_tokens['entities']),
            'lexical_diversity': len(set(word.lower() for word in words if word.isalpha())) / len(words) if words else 0
        }
        
        return stats

# Ejemplo de uso del módulo de tokenización
print("=== MÓDULO 3: TOKENIZACIÓN ===")

# Crear instancia del tokenizador
tokenizer = TextTokenizer()

# Texto de ejemplo sobre medio ambiente
sample_text = """
El cambio climático es uno de los mayores desafíos ambientales de nuestro tiempo. 
La deforestación y la contaminación del aire están afectando gravemente los ecosistemas.

Las energías renovables como la solar y eólica ofrecen soluciones sostenibles. 
Es fundamental proteger la biodiversidad y conservar los recursos naturales para las futuras generaciones.
México tiene importantes reservas naturales que debemos preservar.
"""

print("Texto original:")
print(sample_text)
print("\n" + "="*50)

# Tokenización básica
print("1. TOKENIZACIÓN EN ORACIONES:")
sentences = tokenizer.tokenize_sentences(sample_text, 'spacy')
for i, sentence in enumerate(sentences, 1):
    print(f"   {i}. {sentence}")

print(f"\n2. TOKENIZACIÓN EN PALABRAS (primeras 20):")
words = tokenizer.tokenize_words(sample_text, 'spacy')
print(f"   {words[:20]}...")
print(f"   Total de palabras: {len(words)}")

print(f"\n3. TOKENIZACIÓN EN PÁRRAFOS:")
paragraphs = tokenizer.tokenize_paragraphs(sample_text)
for i, paragraph in enumerate(paragraphs, 1):
    print(f"   Párrafo {i}: {paragraph[:50]}...")

# Tokenización avanzada
print(f"\n4. TOKENIZACIÓN AVANZADA:")
advanced = tokenizer.advanced_tokenization(sample_text)
print(f"   Total de tokens: {advanced['total_tokens']}")
print(f"   Entidades encontradas: {advanced['entities']}")

# Agrupación por categorías gramaticales
print(f"\n5. AGRUPACIÓN POR CATEGORÍAS GRAMATICALES:")
pos_groups = tokenizer.tokenize_by_pos(sample_text)
for category, terms in pos_groups.items():
    if terms:
        print(f"   {category.capitalize()}: {terms[:5]}{'...' if len(terms) > 5 else ''}")

# Términos ambientales
print(f"\n6. TÉRMINOS AMBIENTALES IDENTIFICADOS:")
env_terms = tokenizer.extract_environmental_terms(sample_text)
for category, terms in env_terms.items():
    if terms:
        print(f"   {category.replace('_', ' ').title()}: {terms}")

# Estadísticas completas
print(f"\n7. ESTADÍSTICAS DE TOKENIZACIÓN:")
stats = tokenizer.tokenization_statistics(sample_text)
for key, value in stats.items():
    if isinstance(value, float):
        print(f"   {key.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"   {key.replace('_', ' ').title()}: {value}")


=== MÓDULO 3: TOKENIZACIÓN ===
Texto original:

El cambio climático es uno de los mayores desafíos ambientales de nuestro tiempo. 
La deforestación y la contaminación del aire están afectando gravemente los ecosistemas.

Las energías renovables como la solar y eólica ofrecen soluciones sostenibles. 
Es fundamental proteger la biodiversidad y conservar los recursos naturales para las futuras generaciones.
México tiene importantes reservas naturales que debemos preservar.


1. TOKENIZACIÓN EN ORACIONES:
   1. El cambio climático es uno de los mayores desafíos ambientales de nuestro tiempo.
   2. La deforestación y la contaminación del aire están afectando gravemente los ecosistemas.
   3. Las energías renovables como la solar y eólica ofrecen soluciones sostenibles.
   4. Es fundamental proteger la biodiversidad y conservar los recursos naturales para las futuras generaciones.
   5. México tiene importantes reservas naturales que debemos preservar.

2. TOKENIZACIÓN EN PALABRAS (primeras 

Modulo 4: Normalizacion

In [44]:
# ===== MÓDULO 4: NORMALIZACIÓN =====

class TextNormalizer:
    """
    Módulo para la normalización del texto.
    Convierte texto a formas estándar: minúsculas, eliminación de acentos, 
    expansión de contracciones, normalización de números, etc.
    """
    
    def __init__(self):
        # Diccionario de contracciones en español
        self.contractions = {
            'del': 'de el',
            'al': 'a el',
            'pa': 'para',
            'pal': 'para el',
            'pá': 'para',
            'q': 'que',
            'xq': 'porque',
            'porq': 'porque',
            'x': 'por',
            'tb': 'también',
            'tmb': 'también',
            'tbn': 'también',
            'pq': 'porque',
            'xk': 'porque',
            'k': 'que',
            'dnd': 'donde',
            'dónde': 'donde',
            'cuándo': 'cuando',
            'cómo': 'como',
            'qué': 'que',
            'cuál': 'cual',
            'cuáles': 'cuales',
            'quién': 'quien',
            'quiénes': 'quienes'
        }
        
        # Abreviaciones comunes
        self.abbreviations = {
            'dr.': 'doctor',
            'dra.': 'doctora',
            'sr.': 'señor',
            'sra.': 'señora',
            'srta.': 'señorita',
            'prof.': 'profesor',
            'profa.': 'profesora',
            'ing.': 'ingeniero',
            'lic.': 'licenciado',
            'etc.': 'etcétera',
            'vs.': 'versus',
            'ej.': 'ejemplo',
            'p.ej.': 'por ejemplo',
            'i.e.': 'es decir',
            'e.g.': 'por ejemplo',
            'aprox.': 'aproximadamente',
            'máx.': 'máximo',
            'mín.': 'mínimo',
            'kg.': 'kilogramos',
            'km.': 'kilómetros',
            'm.': 'metros',
            'cm.': 'centímetros',
            'mm.': 'milímetros',
            'co2': 'dióxido de carbono',
            'ong': 'organización no gubernamental',
            'onu': 'organización de las naciones unidas'
        }
        
        # Números escritos
        self.number_words = {
            'cero': '0', 'uno': '1', 'dos': '2', 'tres': '3', 'cuatro': '4',
            'cinco': '5', 'seis': '6', 'siete': '7', 'ocho': '8', 'nueve': '9',
            'diez': '10', 'once': '11', 'doce': '12', 'trece': '13', 'catorce': '14',
            'quince': '15', 'dieciséis': '16', 'diecisiete': '17', 'dieciocho': '18',
            'diecinueve': '19', 'veinte': '20', 'treinta': '30', 'cuarenta': '40',
            'cincuenta': '50', 'sesenta': '60', 'setenta': '70', 'ochenta': '80',
            'noventa': '90', 'cien': '100', 'mil': '1000', 'millón': '1000000'
        }
        
        # Términos ambientales para normalización
        self.env_normalizations = {
            'co2': 'dióxido de carbono',
            'co₂': 'dióxido de carbono',
            'ch4': 'metano',
            'ch₄': 'metano',
            'n2o': 'óxido nitroso',
            'n₂o': 'óxido nitroso',
            'ghg': 'gases de efecto invernadero',
            'gei': 'gases de efecto invernadero',
            'renewable energy': 'energía renovable',
            'green house': 'efecto invernadero',
            'global warming': 'calentamiento global',
            'climate change': 'cambio climático',
            'sustainable development': 'desarrollo sostenible',
            'carbon footprint': 'huella de carbono',
            'biodiversity': 'biodiversidad',
            'ecosystem': 'ecosistema',
            'deforestation': 'deforestación',
            'reforestation': 'reforestación'
        }
    
    def to_lowercase(self, tokens: List[str]) -> List[str]:
        """Convierte tokens a minúsculas"""
        return [token.lower() for token in tokens]
    
    def remove_accents(self, tokens: List[str]) -> List[str]:
        """
        Elimina acentos y diacríticos de los tokens
        """
        normalized_tokens = []
        for token in tokens:
            # Normalizar usando NFD (Canonical Decomposition)
            text_nfd = unicodedata.normalize('NFD', token)
            # Filtrar caracteres diacríticos
            text_without_accents = ''.join(
                char for char in text_nfd 
                if unicodedata.category(char) != 'Mn'
            )
            normalized_tokens.append(text_without_accents)
        return normalized_tokens
    
    def expand_contractions(self, tokens: List[str]) -> List[str]:
        """
        Expande contracciones comunes en español
        """
        expanded_tokens = []
        for token in tokens:
            token_lower = token.lower()
            # Buscar contracción exacta
            if token_lower in self.contractions:
                # Dividir la expansión en tokens si contiene espacios
                expansion = self.contractions[token_lower]
                if ' ' in expansion:
                    expanded_tokens.extend(expansion.split())
                else:
                    expanded_tokens.append(expansion)
            else:
                expanded_tokens.append(token)
        return expanded_tokens
    
    def expand_abbreviations(self, tokens: List[str]) -> List[str]:
        """
        Expande abreviaciones comunes
        """
        expanded_tokens = []
        for token in tokens:
            token_lower = token.lower()
            # Verificar si el token es una abreviación
            if token_lower in self.abbreviations:
                expansion = self.abbreviations[token_lower]
                if ' ' in expansion:
                    expanded_tokens.extend(expansion.split())
                else:
                    expanded_tokens.append(expansion)
            else:
                expanded_tokens.append(token)
        return expanded_tokens
    
    def normalize_numbers(self, tokens: List[str], strategy: str = 'keep') -> List[str]:
        """
        Normaliza números en los tokens
        
        Args:
            tokens: Lista de tokens
            strategy: 'keep', 'remove', 'words_to_digits', 'digits_to_words'
        """
        if strategy == 'remove':
            # Eliminar todos los tokens que son números
            return [token for token in tokens if not token.isdigit()]
        
        elif strategy == 'words_to_digits':
            # Convertir números escritos a dígitos
            normalized_tokens = []
            for token in tokens:
                token_lower = token.lower()
                if token_lower in self.number_words:
                    normalized_tokens.append(self.number_words[token_lower])
                else:
                    normalized_tokens.append(token)
            return normalized_tokens
        
        elif strategy == 'digits_to_words':
            # Convertir dígitos simples a palabras (0-20)
            digit_to_word = {v: k for k, v in self.number_words.items() if int(v) <= 20}
            normalized_tokens = []
            for token in tokens:
                if token.isdigit() and token in digit_to_word:
                    normalized_tokens.append(digit_to_word[token])
                else:
                    normalized_tokens.append(token)
            return normalized_tokens
        
        else:  # 'keep'
            return tokens
    
    def normalize_case_patterns(self, tokens: List[str]) -> List[str]:
        """
        Normaliza patrones de mayúsculas y minúsculas
        """
        normalized_tokens = []
        for token in tokens:
            if token.isupper() and len(token) > 1:
                # Si toda la palabra está en mayúsculas, convertir a minúsculas
                normalized_tokens.append(token.lower())
            elif token.islower():
                # Si está en minúsculas, mantener
                normalized_tokens.append(token)
            else:
                # Casos mixtos, mantener como está
                normalized_tokens.append(token)
        return normalized_tokens
    
    def normalize_environmental_terms(self, tokens: List[str]) -> List[str]:
        """
        Normaliza términos específicos del dominio ambiental
        """
        # Primero, reconstruir el texto para buscar términos compuestos
        text = ' '.join(tokens).lower()
        
        # Aplicar normalizaciones
        for term, normalized in self.env_normalizations.items():
            text = re.sub(r'\b' + re.escape(term) + r'\b', normalized, text)
        
        # Volver a tokenizar
        return text.split()
    
    def normalize_tokens(self, tokens: List[str], options: Dict = None) -> Dict:
        """
        Realiza normalización completa de los tokens
        
        Args:
            tokens: Lista de tokens a normalizar
            options: Diccionario con opciones de normalización
        
        Returns:
            Diccionario con tokens normalizados y estadísticas
        """
        if options is None:
            options = {
                'to_lowercase': True,
                'remove_accents': True,
                'expand_contractions': True,
                'expand_abbreviations': True,
                'normalize_numbers': 'keep',
                'normalize_case_patterns': True,
                'normalize_environmental_terms': True
            }
        
        original_tokens = tokens.copy()
        steps_applied = []
        
        # Aplicar normalizaciones según opciones
        if options.get('normalize_case_patterns', True):
            tokens = self.normalize_case_patterns(tokens)
            steps_applied.append('case_patterns')
        
        if options.get('to_lowercase', True):
            tokens = self.to_lowercase(tokens)
            steps_applied.append('lowercase')
        
        if options.get('expand_contractions', True):
            tokens = self.expand_contractions(tokens)
            steps_applied.append('contractions')
        
        if options.get('expand_abbreviations', True):
            tokens = self.expand_abbreviations(tokens)
            steps_applied.append('abbreviations')
        
        if options.get('normalize_environmental_terms', True):
            tokens = self.normalize_environmental_terms(tokens)
            steps_applied.append('environmental_terms')
        
        if options.get('normalize_numbers', 'keep') != 'keep':
            tokens = self.normalize_numbers(tokens, options['normalize_numbers'])
            steps_applied.append('numbers')
        
        if options.get('remove_accents', True):
            tokens = self.remove_accents(tokens)
            steps_applied.append('accents')
        
        return {
            'original_tokens': original_tokens,
            'normalized_tokens': tokens,
            'original_count': len(original_tokens),
            'normalized_count': len(tokens),
            'token_count_change': len(tokens) - len(original_tokens),
            'steps_applied': steps_applied,
            'normalization_options': options
        }
    
    def comprehensive_normalization(self, text: str, options: Dict = None) -> Dict:
        """
        Realiza normalización completa del texto (para compatibilidad con versiones anteriores)
        
        Args:
            text: Texto a normalizar
            options: Diccionario con opciones de normalización
        
        Returns:
            Diccionario con texto normalizado y estadísticas
        """
        # Tokenizar el texto
        tokens = text.split()
        
        # Normalizar los tokens
        result = self.normalize_tokens(tokens, options)
        
        # Reconstruir el texto normalizado
        normalized_text = ' '.join(result['normalized_tokens'])
        
        return {
            'original_text': text,
            'normalized_text': normalized_text,
            'original_word_count': result['original_count'],
            'normalized_word_count': result['normalized_count'],
            'word_count_change': result['token_count_change'],
            'steps_applied': result['steps_applied'],
            'normalization_options': result['normalization_options']
        }
    
    def normalize_tokenized_text(self, tokenization_result: Dict, options: Dict = None) -> Dict:
        """
        Normaliza el resultado de la tokenización
        
        Args:
            tokenization_result: Resultado del módulo de tokenización
            options: Opciones de normalización
        
        Returns:
            Diccionario con resultados normalizados
        """
        # Extraer tokens del resultado de tokenización
        if 'tokens' in tokenization_result:
            # Si es resultado de advanced_tokenization
            tokens = [token['text'] for token in tokenization_result['tokens']]
        elif isinstance(tokenization_result, list):
            # Si es una lista simple de tokens
            tokens = tokenization_result
        else:
            # Intentar extraer palabras
            tokens = tokenization_result.get('words', [])
            if not tokens:
                raise ValueError("No se pudieron extraer tokens del resultado de tokenización")
        
        # Normalizar los tokens
        normalization_result = self.normalize_tokens(tokens, options)
        
        # Si el resultado original incluye información POS, mantenerla
        if 'tokens' in tokenization_result and isinstance(tokenization_result['tokens'], list):
            # Crear un mapeo de tokens originales a normalizados
            # (esto es una aproximación, ya que la normalización puede cambiar el número de tokens)
            normalized_tokens_info = []
            
            # Intentar preservar información lingüística
            for i, token_info in enumerate(tokenization_result['tokens']):
                if i < len(normalization_result['normalized_tokens']):
                    normalized_token = normalization_result['normalized_tokens'][i]
                    normalized_tokens_info.append({
                        **token_info,
                        'original_text': token_info['text'],
                        'text': normalized_token,
                        'normalized': True
                    })
            
            normalization_result['normalized_tokens_info'] = normalized_tokens_info
        
        return normalization_result

# Ejemplo de uso del módulo de normalización con tokens
print("=== MÓDULO 4: NORMALIZACIÓN (CON TOKENS) ===")

# Crear instancias de tokenizador y normalizador
tokenizer = TextTokenizer()
normalizer = TextNormalizer()

# Texto de ejemplo con varios problemas de normalización
sample_text = """
El Dr. García explicó q el CO2 y otros GEI están causando el CALENTAMIENTO GLOBAL.
Las ONGs trabajan pa proteger la biodiversidad. Aprox. el 30% de los bosques
han sido deforestados. ¿Cuándo tomaremos acción? ¡Es urgente!

Necesitamos energías renovables como la solar y eólica... También debemos
reducir nuestro "carbon footprint" y promover el "sustainable development".
Tres millones de hectáreas se pierden cada año.
"""

print("Texto original:")
print(sample_text)
print("\n" + "="*60)

# Tokenizar primero
print("1. TOKENIZACIÓN:")
tokens = tokenizer.tokenize_words(sample_text, 'spacy')
print(f"Tokens originales: {tokens[:20]}...")
print(f"Total de tokens: {len(tokens)}")

# Normalizar los tokens
print("\n2. NORMALIZACIÓN DE TOKENS:")
normalization_result = normalizer.normalize_tokens(tokens)
print(f"Tokens normalizados: {normalization_result['normalized_tokens'][:20]}...")
print(f"Total de tokens normalizados: {normalization_result['normalized_count']}")
print(f"Cambio en cantidad de tokens: {normalization_result['token_count_change']}")
print(f"Pasos aplicados: {', '.join(normalization_result['steps_applied'])}")

# Normalizar resultado de tokenización avanzada
print("\n3. NORMALIZACIÓN DE TOKENIZACIÓN AVANZADA:")
advanced_tokens = tokenizer.advanced_tokenization(sample_text)
advanced_normalization = normalizer.normalize_tokenized_text(advanced_tokens)

print(f"Tokens avanzados normalizados: {len(advanced_normalization['normalized_tokens'])}")
if 'normalized_tokens_info' in advanced_normalization:
    print("Ejemplos de tokens con información lingüística preservada:")
    for token_info in advanced_normalization['normalized_tokens_info'][:5]:
        print(f"  Original: '{token_info['original_text']}' → Normalizado: '{token_info['text']}' (POS: {token_info['pos']})")

# Ejemplo con opciones personalizadas
print("\n" + "="*40)
print("4. NORMALIZACIÓN CON OPCIONES PERSONALIZADAS:")

custom_options = {
    'to_lowercase': True,
    'remove_accents': False,  # Mantener acentos
    'expand_contractions': True,
    'expand_abbreviations': True,
    'normalize_numbers': 'words_to_digits',  # Convertir números escritos a dígitos
    'normalize_case_patterns': True,
    'normalize_environmental_terms': True
}

custom_result = normalizer.normalize_tokens(tokens, custom_options)
print("Tokens con normalización personalizada:")
print(f"'{custom_result['normalized_tokens'][:20]}...'")
print(f"Pasos aplicados: {', '.join(custom_result['steps_applied'])}")

# Integración completa de tokenización y normalización
print("\n" + "="*60)
print("5. INTEGRACIÓN COMPLETA:")

# Tokenizar por oraciones
sentences = tokenizer.tokenize_sentences(sample_text, 'spacy')
print(f"Oraciones tokenizadas: {len(sentences)}")

# Normalizar cada oración
normalized_sentences = []
for sentence in sentences:
    # Tokenizar la oración en palabras
    sentence_tokens = tokenizer.tokenize_words(sentence, 'spacy')
    # Normalizar los tokens
    normalized_result = normalizer.normalize_tokens(sentence_tokens)
    # Reconstruir la oración normalizada
    normalized_sentence = ' '.join(normalized_result['normalized_tokens'])
    normalized_sentences.append(normalized_sentence)

print("Oraciones normalizadas:")
for i, sentence in enumerate(normalized_sentences[:3], 1):
    print(f"  {i}. {sentence}")

# Reconstruir texto completo normalizado
final_text = ' '.join(normalized_sentences)
print("\nTexto final normalizado:")
print(f"'{final_text}'")


=== MÓDULO 4: NORMALIZACIÓN (CON TOKENS) ===
Texto original:

El Dr. García explicó q el CO2 y otros GEI están causando el CALENTAMIENTO GLOBAL.
Las ONGs trabajan pa proteger la biodiversidad. Aprox. el 30% de los bosques
han sido deforestados. ¿Cuándo tomaremos acción? ¡Es urgente!

Necesitamos energías renovables como la solar y eólica... También debemos
reducir nuestro "carbon footprint" y promover el "sustainable development".
Tres millones de hectáreas se pierden cada año.


1. TOKENIZACIÓN:
Tokens originales: ['El', 'Dr.', 'García', 'explicó', 'q', 'el', 'CO2', 'y', 'otros', 'GEI', 'están', 'causando', 'el', 'CALENTAMIENTO', 'GLOBAL', '.', 'Las', 'ONGs', 'trabajan', 'pa']...
Total de tokens: 78

2. NORMALIZACIÓN DE TOKENS:
Tokens normalizados: ['el', 'doctor', 'garcia', 'explico', 'que', 'el', 'dioxido', 'de', 'carbono', 'y', 'otros', 'gases', 'de', 'efecto', 'invernadero', 'estan', 'causando', 'el', 'calentamiento', 'global']...
Total de tokens normalizados: 84
Cambio en cantida

Modulo 5: Eliminacion de Ruido

In [45]:
# ===== MÓDULO 5: ELIMINACIÓN DE RUIDO =====

class NoiseRemover:
    """
    Módulo para la eliminación de ruido del texto.
    Elimina stopwords, palabras muy frecuentes/raras, contenido irrelevante
    y ruido específico del dominio.
    """
    
    def __init__(self):
        # Stopwords básicas en español
        self.spanish_stopwords = set(stopwords.words('spanish'))
        
        # Stopwords adicionales personalizadas
        self.custom_stopwords = {
            # Conectores y muletillas
            'pues', 'bueno', 'entonces', 'así', 'ahora', 'luego', 'después',
            'antes', 'mientras', 'durante', 'mediante', 'según', 'incluso',
            'además', 'también', 'tampoco', 'sino', 'aunque', 'sin embargo',
            'no obstante', 'por tanto', 'por consiguiente', 'en consecuencia',
            
            # Palabras de relleno
            'cosa', 'cosas', 'algo', 'nada', 'todo', 'todos', 'todas',
            'mucho', 'muchos', 'muchas', 'poco', 'pocos', 'pocas',
            'bastante', 'demasiado', 'suficiente',
            
            # Palabras muy generales
            'forma', 'manera', 'modo', 'tipo', 'tipos', 'clase', 'clases',
            'parte', 'partes', 'lado', 'lados', 'vez', 'veces', 'momento',
            'momentos', 'tiempo', 'tiempos', 'lugar', 'lugares', 'caso', 'casos'
        }
        
        # Stopwords específicas para contexto ambiental (palabras muy comunes que no aportan)
        self.environmental_stopwords = {
            'tema', 'temas', 'problema', 'problemas', 'situación', 'situaciones',
            'aspecto', 'aspectos', 'factor', 'factores', 'elemento', 'elementos',
            'punto', 'puntos', 'cuestión', 'cuestiones', 'asunto', 'asuntos'
        }
        
        # Combinar todos los stopwords
        self.all_stopwords = (self.spanish_stopwords | 
                             self.custom_stopwords | 
                             self.environmental_stopwords)
        
        # Patrones de ruido común
        self.noise_patterns = [
            r'\b\w{1,2}\b',  # Palabras muy cortas (1-2 caracteres)
            r'\b\d+\b',      # Números sueltos (opcional)
            r'[^\w\s]',      # Signos de puntuación sueltos
            r'\b[a-zA-Z]\b', # Letras sueltas
        ]
        
        # Palabras de ruido específicas
        self.noise_words = {
            'mm', 'hmm', 'eh', 'ah', 'oh', 'uh', 'um', 'er',
            'ok', 'okay', 'si', 'no', 'ya', 'je', 'ja', 'jaja', 'jeje',
            'etc', 'etcetera', 'bla', 'blah'
        }
    
    def remove_stopwords(self, text: str, custom_stopwords: set = None) -> str:
        """
        Elimina stopwords del texto
        
        Args:
            text: Texto a procesar
            custom_stopwords: Stopwords adicionales personalizadas
        
        Returns:
            Texto sin stopwords
        """
        words = text.split()
        stopwords_to_use = self.all_stopwords.copy()
        
        if custom_stopwords:
            stopwords_to_use.update(custom_stopwords)
        
        filtered_words = [
            word for word in words 
            if word.lower() not in stopwords_to_use
        ]
        
        return ' '.join(filtered_words)
    
    def remove_by_frequency(self, text: str, min_freq: int = 2, max_freq_ratio: float = 0.1) -> Dict:
        """
        Elimina palabras muy raras o muy frecuentes
        
        Args:
            text: Texto a procesar
            min_freq: Frecuencia mínima para mantener una palabra
            max_freq_ratio: Ratio máximo de frecuencia (ej: 0.1 = 10% del total)
        
        Returns:
            Diccionario con texto filtrado y estadísticas
        """
        words = text.split()
        word_freq = {}
        
        # Contar frecuencias
        for word in words:
            word_lower = word.lower()
            word_freq[word_lower] = word_freq.get(word_lower, 0) + 1
        
        total_words = len(words)
        max_freq = int(total_words * max_freq_ratio)
        
        # Identificar palabras a eliminar
        rare_words = {word for word, freq in word_freq.items() if freq < min_freq}
        frequent_words = {word for word, freq in word_freq.items() if freq > max_freq}
        words_to_remove = rare_words | frequent_words
        
        # Filtrar palabras
        filtered_words = [
            word for word in words 
            if word.lower() not in words_to_remove
        ]
        
        return {
            'filtered_text': ' '.join(filtered_words),
            'original_word_count': len(words),
            'filtered_word_count': len(filtered_words),
            'rare_words_removed': len(rare_words),
            'frequent_words_removed': len(frequent_words),
            'rare_words': list(rare_words)[:10],  # Mostrar solo las primeras 10
            'frequent_words': list(frequent_words)
        }
    
    def remove_noise_patterns(self, text: str) -> str:
        """
        Elimina patrones de ruido usando regex
        """
        for pattern in self.noise_patterns:
            text = re.sub(pattern, ' ', text)
        
        # Normalizar espacios
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def remove_noise_words(self, text: str) -> str:
        """
        Elimina palabras de ruido específicas
        """
        words = text.split()
        filtered_words = [
            word for word in words 
            if word.lower() not in self.noise_words
        ]
        return ' '.join(filtered_words)
    
    def remove_short_words(self, text: str, min_length: int = 3) -> str:
        """
        Elimina palabras muy cortas
        """
        words = text.split()
        filtered_words = [
            word for word in words 
            if len(word) >= min_length
        ]
        return ' '.join(filtered_words)
    
    def remove_non_alphabetic(self, text: str, keep_numbers: bool = False) -> str:
        """
        Elimina tokens que no son alfabéticos
        """
        words = text.split()
        if keep_numbers:
            # Mantener palabras alfabéticas y números
            filtered_words = [
                word for word in words 
                if word.isalpha() or word.isdigit()
            ]
        else:
            # Solo mantener palabras alfabéticas
            filtered_words = [
                word for word in words 
                if word.isalpha()
            ]
        
        return ' '.join(filtered_words)
    
    def remove_by_pos(self, text: str, pos_to_remove: List[str] = None) -> str:
        """
        Elimina palabras según su categoría gramatical
        
        Args:
            text: Texto a procesar
            pos_to_remove: Lista de POS tags a eliminar
        """
        if pos_to_remove is None:
            # Por defecto, eliminar determinantes, preposiciones, conjunciones
            pos_to_remove = ['DET', 'ADP', 'CONJ', 'CCONJ', 'SCONJ']
        
        doc = nlp(text)
        filtered_words = [
            token.text for token in doc 
            if token.pos_ not in pos_to_remove and not token.is_space
        ]
        
        return ' '.join(filtered_words)
    
    def remove_environmental_noise(self, text: str) -> str:
        """
        Elimina ruido específico del dominio ambiental
        """
        # Palabras muy generales en contexto ambiental que no aportan información específica
        env_noise = {
            'importante', 'necesario', 'fundamental', 'esencial', 'básico',
            'general', 'específico', 'particular', 'especial', 'normal',
            'actual', 'presente', 'futuro', 'pasado', 'nuevo', 'viejo',
            'grande', 'pequeño', 'mayor', 'menor', 'mejor', 'peor',
            'bueno', 'malo', 'positivo', 'negativo', 'principal', 'secundario'
        }
        
        words = text.split()
        filtered_words = [
            word for word in words 
            if word.lower() not in env_noise
        ]
        
        return ' '.join(filtered_words)
    
    def advanced_noise_removal(self, text: str) -> Dict:
        """
        Eliminación avanzada de ruido usando spaCy
        """
        doc = nlp(text)
        
        # Criterios para mantener tokens
        kept_tokens = []
        removed_tokens = []
        
        for token in doc:
            # Criterios para eliminar
            should_remove = (
                token.is_stop or           # Es stopword
                token.is_punct or          # Es puntuación
                token.is_space or          # Es espacio
                len(token.text) < 3 or     # Muy corto
                not token.is_alpha or      # No es alfabético
                token.pos_ in ['DET', 'ADP', 'CONJ', 'CCONJ'] or  # POS irrelevantes
                token.text.lower() in self.noise_words  # Palabras de ruido
            )
            
            if should_remove:
                removed_tokens.append({
                    'text': token.text,
                    'reason': self._get_removal_reason(token)
                })
            else:
                kept_tokens.append(token.lemma_.lower())
        
        return {
            'cleaned_text': ' '.join(kept_tokens),
            'original_tokens': len(doc),
            'kept_tokens': len(kept_tokens),
            'removed_tokens': len(removed_tokens),
            'removal_details': removed_tokens[:20]  # Mostrar solo los primeros 20
        }
    
    def _get_removal_reason(self, token) -> str:
        """Determina la razón por la cual se elimina un token"""
        if token.is_stop:
            return 'stopword'
        elif token.is_punct:
            return 'punctuation'
        elif token.is_space:
            return 'whitespace'
        elif len(token.text) < 3:
            return 'too_short'
        elif not token.is_alpha:
            return 'non_alphabetic'
        elif token.pos_ in ['DET', 'ADP', 'CONJ', 'CCONJ']:
            return 'irrelevant_pos'
        elif token.text.lower() in self.noise_words:
            return 'noise_word'
        else:
            return 'other'
    
    def comprehensive_noise_removal(self, text: str, options: Dict = None) -> Dict:
        """
        Eliminación completa de ruido con múltiples estrategias
        
        Args:
            text: Texto a limpiar
            options: Opciones de limpieza
        
        Returns:
            Diccionario con texto limpio y estadísticas
        """
        if options is None:
            options = {
                'remove_stopwords': True,
                'remove_short_words': True,
                'min_word_length': 3,
                'remove_noise_words': True,
                'remove_by_frequency': True,
                'min_frequency': 2,
                'max_frequency_ratio': 0.15,
                'remove_non_alphabetic': True,
                'keep_numbers': False,
                'remove_by_pos': True,
                'remove_environmental_noise': True,
                'use_advanced_removal': True
            }
        
        original_text = text
        processing_steps = []
        
        # Aplicar eliminaciones según opciones
        if options.get('remove_stopwords', True):
            text = self.remove_stopwords(text)
            processing_steps.append('stopwords')
        
        if options.get('remove_noise_words', True):
            text = self.remove_noise_words(text)
            processing_steps.append('noise_words')
        
        if options.get('remove_short_words', True):
            min_length = options.get('min_word_length', 3)
            text = self.remove_short_words(text, min_length)
            processing_steps.append('short_words')
        
        if options.get('remove_environmental_noise', True):
            text = self.remove_environmental_noise(text)
            processing_steps.append('environmental_noise')
        
        if options.get('remove_non_alphabetic', True):
            keep_nums = options.get('keep_numbers', False)
            text = self.remove_non_alphabetic(text, keep_nums)
            processing_steps.append('non_alphabetic')
        
        if options.get('remove_by_pos', True):
            text = self.remove_by_pos(text)
            processing_steps.append('pos_filtering')
        
        # Eliminación por frecuencia (al final para tener estadísticas correctas)
        freq_result = None
        if options.get('remove_by_frequency', True):
            min_freq = options.get('min_frequency', 2)
            max_ratio = options.get('max_frequency_ratio', 0.15)
            freq_result = self.remove_by_frequency(text, min_freq, max_ratio)
            text = freq_result['filtered_text']
            processing_steps.append('frequency_filtering')
        
        # Eliminación avanzada (opcional)
        advanced_result = None
        if options.get('use_advanced_removal', True):
            advanced_result = self.advanced_noise_removal(text)
            text = advanced_result['cleaned_text']
            processing_steps.append('advanced_removal')
        
        # Calcular estadísticas finales
        original_words = len(original_text.split())
        final_words = len(text.split())
        reduction_percentage = ((original_words - final_words) / original_words) * 100 if original_words > 0 else 0
        
        result = {
            'original_text': original_text,
            'cleaned_text': text,
            'original_word_count': original_words,
            'final_word_count': final_words,
            'words_removed': original_words - final_words,
            'reduction_percentage': round(reduction_percentage, 2),
            'processing_steps': processing_steps,
            'options_used': options
        }
        
        # Agregar resultados específicos si están disponibles
        if freq_result:
            result['frequency_analysis'] = freq_result
        if advanced_result:
            result['advanced_analysis'] = advanced_result
        
        return result

# Ejemplo de uso del módulo de eliminación de ruido
print("=== MÓDULO 5: ELIMINACIÓN DE RUIDO ===")

# Crear instancia del eliminador de ruido
noise_remover = NoiseRemover()

# Texto de ejemplo con mucho ruido
noisy_text = """
Bueno, pues, el tema del cambio climático es, eh, muy importante y fundamental.
La cosa es que, mm, tenemos muchos problemas ambientales que son, así, bastante serios.
Ok, entonces, la deforestación y la contaminación del aire son, pues, cuestiones
que debemos, eh, abordar de manera urgente. ¿No? Sí, es algo muy necesario.

Los aspectos más importantes incluyen: a) energías renovables, b) conservación,
c) sostenibilidad. Etc, etc. Jaja, pero en serio, es un asunto muy general
que requiere, mm, acciones específicas y particulares para el futuro.
"""

print("Texto original con ruido:")
print(noisy_text)
print("\n" + "="*60)

# Eliminación paso a paso
print("1. ELIMINACIÓN DE STOPWORDS:")
no_stopwords = noise_remover.remove_stopwords(noisy_text)
print(no_stopwords[:150] + "...")

print("\n2. ELIMINACIÓN DE PALABRAS DE RUIDO:")
no_noise_words = noise_remover.remove_noise_words(no_stopwords)
print(no_noise_words[:150] + "...")

print("\n3. ELIMINACIÓN DE PALABRAS CORTAS:")
no_short = noise_remover.remove_short_words(no_noise_words, min_length=3)
print(no_short[:150] + "...")

print("\n4. ELIMINACIÓN DE RUIDO AMBIENTAL:")
no_env_noise = noise_remover.remove_environmental_noise(no_short)
print(no_env_noise[:150] + "...")

# Análisis por frecuencia
print("\n5. ANÁLISIS POR FRECUENCIA:")
freq_analysis = noise_remover.remove_by_frequency(no_env_noise, min_freq=2, max_freq_ratio=0.1)
print(f"Palabras raras eliminadas: {freq_analysis['rare_words_removed']}")
print(f"Palabras muy frecuentes eliminadas: {freq_analysis['frequent_words_removed']}")
print(f"Ejemplos de palabras raras: {freq_analysis['rare_words']}")

# Eliminación completa
print("\n" + "="*60)
print("ELIMINACIÓN COMPLETA DE RUIDO:")

result = noise_remover.comprehensive_noise_removal(noisy_text)

print("Texto completamente limpio:")
print(f"'{result['cleaned_text']}'")

print(f"\nEstadísticas de limpieza:")
print(f"- Palabras originales: {result['original_word_count']}")
print(f"- Palabras finales: {result['final_word_count']}")
print(f"- Palabras eliminadas: {result['words_removed']}")
print(f"- Reducción: {result['reduction_percentage']}%")
print(f"- Pasos aplicados: {', '.join(result['processing_steps'])}")

# Eliminación avanzada con spaCy
print("\n" + "="*40)
print("ANÁLISIS AVANZADO DE ELIMINACIÓN:")

advanced_result = noise_remover.advanced_noise_removal(noisy_text)
print(f"Tokens originales: {advanced_result['original_tokens']}")
print(f"Tokens mantenidos: {advanced_result['kept_tokens']}")
print(f"Tokens eliminados: {advanced_result['removed_tokens']}")

print("\nDetalles de eliminación (primeros 10):")
for detail in advanced_result['removal_details'][:10]:
    print(f"  '{detail['text']}' -> {detail['reason']}")


=== MÓDULO 5: ELIMINACIÓN DE RUIDO ===
Texto original con ruido:

Bueno, pues, el tema del cambio climático es, eh, muy importante y fundamental.
La cosa es que, mm, tenemos muchos problemas ambientales que son, así, bastante serios.
Ok, entonces, la deforestación y la contaminación del aire son, pues, cuestiones
que debemos, eh, abordar de manera urgente. ¿No? Sí, es algo muy necesario.

Los aspectos más importantes incluyen: a) energías renovables, b) conservación,
c) sostenibilidad. Etc, etc. Jaja, pero en serio, es un asunto muy general
que requiere, mm, acciones específicas y particulares para el futuro.


1. ELIMINACIÓN DE STOPWORDS:
Bueno, pues, cambio climático es, eh, importante fundamental. que, mm, ambientales son, así, serios. Ok, entonces, deforestación contaminación aire so...

2. ELIMINACIÓN DE PALABRAS DE RUIDO:
Bueno, pues, cambio climático es, eh, importante fundamental. que, mm, ambientales son, así, serios. Ok, entonces, deforestación contaminación aire so...

3. EL

Modulo 6: Lematizacion y Stemming

In [46]:
# ===== MÓDULO 6: LEMATIZACIÓN Y STEMMING =====

class TextLemmatizer:
    """
    Módulo para lematización y stemming del texto.
    Reduce las palabras a su forma base usando diferentes estrategias.
    Incluye evaluación comparativa entre métodos.
    """
    
    def __init__(self):
        # Configurar herramientas
        self.nlp = nlp  # spaCy model
        self.stemmer = SnowballStemmer('spanish')
        self.nltk_lemmatizer = WordNetLemmatizer()  # Para comparación
        
        # Diccionario de lemas personalizados para términos ambientales
        self.environmental_lemmas = {
            'contaminaciones': 'contaminación',
            'deforestaciones': 'deforestación',
            'reforestaciones': 'reforestación',
            'sostenibilidades': 'sostenibilidad',
            'biodiversidades': 'biodiversidad',
            'ecosistemas': 'ecosistema',
            'energías': 'energía',
            'renovables': 'renovable',
            'combustibles': 'combustible',
            'emisiones': 'emisión',
            'residuos': 'residuo',
            'desechos': 'desecho',
            'conservaciones': 'conservación',
            'preservaciones': 'preservación',
            'protecciones': 'protección',
            'calentamientos': 'calentamiento',
            'cambios': 'cambio',
            'efectos': 'efecto',
            'impactos': 'impacto',
            'consecuencias': 'consecuencia',
            'soluciones': 'solución',
            'alternativas': 'alternativa',
            'tecnologías': 'tecnología',
            'innovaciones': 'innovación'
        }
        
        # Excepciones de stemming (palabras que no deben ser stemmed)
        self.stemming_exceptions = {
            'gases', 'atlas', 'crisis', 'análisis', 'síntesis',
            'tesis', 'oasis', 'énfasis', 'paréntesis'
        }
    
    def lemmatize_with_spacy(self, text: str) -> Dict:
        """
        Lematización usando spaCy (método recomendado)
        
        Returns:
            Diccionario con texto lematizado y análisis detallado
        """
        doc = self.nlp(text)
        
        lemmatized_tokens = []
        lemmatization_details = []
        
        for token in doc:
            if not token.is_space and not token.is_punct:
                original = token.text
                lemma = token.lemma_.lower()
                
                # Aplicar lemas personalizados si existen
                if original.lower() in self.environmental_lemmas:
                    lemma = self.environmental_lemmas[original.lower()]
                
                lemmatized_tokens.append(lemma)
                
                # Guardar detalles si hay cambio
                if original.lower() != lemma:
                    lemmatization_details.append({
                        'original': original,
                        'lemma': lemma,
                        'pos': token.pos_,
                        'change_type': self._get_change_type(original, lemma)
                    })
        
        return {
            'lemmatized_text': ' '.join(lemmatized_tokens),
            'original_tokens': len([t for t in doc if not t.is_space and not t.is_punct]),
            'lemmatized_tokens': len(lemmatized_tokens),
            'changes_made': len(lemmatization_details),
            'lemmatization_details': lemmatization_details,
            'method': 'spacy'
        }
    
    def stem_with_snowball(self, text: str) -> Dict:
        """
        Stemming usando SnowballStemmer
        
        Returns:
            Diccionario con texto stemmed y análisis detallado
        """
        words = text.split()
        
        stemmed_tokens = []
        stemming_details = []
        
        for word in words:
            if word.isalpha():
                original = word.lower()
                
                # Verificar excepciones
                if original in self.stemming_exceptions:
                    stemmed = original
                else:
                    stemmed = self.stemmer.stem(original)
                
                stemmed_tokens.append(stemmed)
                
                # Guardar detalles si hay cambio
                if original != stemmed:
                    stemming_details.append({
                        'original': word,
                        'stem': stemmed,
                        'change_type': self._get_change_type(original, stemmed)
                    })
            else:
                stemmed_tokens.append(word)
        
        return {
            'stemmed_text': ' '.join(stemmed_tokens),
            'original_tokens': len(words),
            'stemmed_tokens': len(stemmed_tokens),
            'changes_made': len(stemming_details),
            'stemming_details': stemming_details,
            'method': 'snowball'
        }
    
    def hybrid_approach(self, text: str) -> Dict:
        """
        Enfoque híbrido: lematización para sustantivos/adjetivos, stemming para verbos
        """
        doc = self.nlp(text)
        
        processed_tokens = []
        processing_details = []
        
        for token in doc:
            if not token.is_space and not token.is_punct and token.is_alpha:
                original = token.text
                
                # Decidir método según POS
                if token.pos_ in ['NOUN', 'ADJ', 'PROPN']:
                    # Usar lematización para sustantivos y adjetivos
                    processed = token.lemma_.lower()
                    method_used = 'lemmatization'
                    
                    # Aplicar lemas personalizados
                    if original.lower() in self.environmental_lemmas:
                        processed = self.environmental_lemmas[original.lower()]
                        method_used = 'custom_lemma'
                        
                elif token.pos_ in ['VERB', 'AUX']:
                    # Usar stemming para verbos
                    if original.lower() not in self.stemming_exceptions:
                        processed = self.stemmer.stem(original.lower())
                        method_used = 'stemming'
                    else:
                        processed = original.lower()
                        method_used = 'exception'
                else:
                    # Para otras categorías, usar lematización
                    processed = token.lemma_.lower()
                    method_used = 'lemmatization'
                
                processed_tokens.append(processed)
                
                # Guardar detalles si hay cambio
                if original.lower() != processed:
                    processing_details.append({
                        'original': original,
                        'processed': processed,
                        'pos': token.pos_,
                        'method': method_used,
                        'change_type': self._get_change_type(original, processed)
                    })
        
        return {
            'processed_text': ' '.join(processed_tokens),
            'original_tokens': len([t for t in doc if not t.is_space and not t.is_punct and t.is_alpha]),
            'processed_tokens': len(processed_tokens),
            'changes_made': len(processing_details),
            'processing_details': processing_details,
            'method': 'hybrid'
        }
    
    def _get_change_type(self, original: str, processed: str) -> str:
        """Determina el tipo de cambio realizado"""
        if len(processed) < len(original):
            return 'reduction'
        elif len(processed) > len(original):
            return 'expansion'
        elif processed != original.lower():
            return 'transformation'
        else:
            return 'no_change'
    
    def compare_methods(self, text: str) -> Dict:
        """
        Compara los tres métodos de procesamiento
        """
        # Aplicar cada método
        spacy_result = self.lemmatize_with_spacy(text)
        snowball_result = self.stem_with_snowball(text)
        hybrid_result = self.hybrid_approach(text)
        
        # Calcular métricas de comparación
        original_words = set(text.lower().split())
        spacy_words = set(spacy_result['lemmatized_text'].split())
        snowball_words = set(snowball_result['stemmed_text'].split())
        hybrid_words = set(hybrid_result['processed_text'].split())
        
        # Calcular reducción de vocabulario
        vocab_reduction = {
            'original': len(original_words),
            'spacy': len(spacy_words),
            'snowball': len(snowball_words),
            'hybrid': len(hybrid_words)
        }
        
        # Calcular porcentajes de reducción
        reduction_percentages = {}
        for method, vocab_size in vocab_reduction.items():
            if method != 'original':
                reduction_percentages[method] = round(
                    ((vocab_reduction['original'] - vocab_size) / vocab_reduction['original']) * 100, 2
                )
        
        return {
            'results': {
                'spacy': spacy_result,
                'snowball': snowball_result,
                'hybrid': hybrid_result
            },
            'vocabulary_sizes': vocab_reduction,
            'reduction_percentages': reduction_percentages,
            'comparison_summary': {
                'most_aggressive': min(vocab_reduction, key=vocab_reduction.get),
                'most_conservative': max(vocab_reduction, key=vocab_reduction.get),
                'recommended': 'spacy'  # Nuestra recomendación inicial
            }
        }
    
    def evaluate_environmental_terms(self, text: str) -> Dict:
        """
        Evalúa cómo cada método maneja términos ambientales específicos
        """
        # Términos ambientales de prueba
        test_terms = [
            'contaminaciones', 'deforestación', 'sostenibilidad', 'biodiversidad',
            'energías renovables', 'combustibles fósiles', 'emisiones tóxicas',
            'conservación ambiental', 'calentamiento global', 'cambio climático'
        ]
        
        test_text = ' '.join(test_terms)
        
        # Aplicar métodos
        spacy_result = self.lemmatize_with_spacy(test_text)
        snowball_result = self.stem_with_snowball(test_text)
        hybrid_result = self.hybrid_approach(test_text)
        
        return {
            'test_terms': test_terms,
            'spacy_output': spacy_result['lemmatized_text'],
            'snowball_output': snowball_result['stemmed_text'],
            'hybrid_output': hybrid_result['processed_text'],
            'evaluation': {
                'spacy_preserves_meaning': self._check_meaning_preservation(test_terms, spacy_result['lemmatized_text']),
                'snowball_preserves_meaning': self._check_meaning_preservation(test_terms, snowball_result['stemmed_text']),
                'hybrid_preserves_meaning': self._check_meaning_preservation(test_terms, hybrid_result['processed_text'])
            }
        }
    
    def _check_meaning_preservation(self, original_terms: List[str], processed_text: str) -> float:
        """
        Verifica qué porcentaje de términos mantienen su significado reconocible
        """
        processed_words = processed_text.split()
        recognizable_count = 0
        
        for word in processed_words:
            # Verificar si la palabra procesada es reconocible
            # (esto es una aproximación simple)
            if len(word) >= 4 and word.isalpha():
                recognizable_count += 1
        
        return round((recognizable_count / len(processed_words)) * 100, 2) if processed_words else 0
    
    def comprehensive_processing(self, text: str, method: str = 'spacy', options: Dict = None) -> Dict:
        """
        Procesamiento completo con el método seleccionado
        
        Args:
            text: Texto a procesar
            method: 'spacy', 'snowball', 'hybrid', o 'compare'
            options: Opciones adicionales
        """
        if options is None:
            options = {
                'preserve_environmental_terms': True,
                'min_word_length': 2,
                'remove_duplicates': True
            }
        
        if method == 'compare':
            return self.compare_methods(text)
        elif method == 'spacy':
            result = self.lemmatize_with_spacy(text)
        elif method == 'snowball':
            result = self.stem_with_snowball(text)
        elif method == 'hybrid':
            result = self.hybrid_approach(text)
        else:
            raise ValueError("Método debe ser 'spacy', 'snowball', 'hybrid', o 'compare'")
        
        # Aplicar opciones adicionales
        processed_text = result.get('lemmatized_text') or result.get('stemmed_text') or result.get('processed_text')
        
        if options.get('min_word_length', 2) > 1:
            words = processed_text.split()
            words = [w for w in words if len(w) >= options['min_word_length']]
            processed_text = ' '.join(words)
        
        if options.get('remove_duplicates', True):
            words = processed_text.split()
            # Mantener orden pero eliminar duplicados
            seen = set()
            unique_words = []
            for word in words:
                if word not in seen:
                    seen.add(word)
                    unique_words.append(word)
            processed_text = ' '.join(unique_words)
        
        # Actualizar resultado
        result['final_processed_text'] = processed_text
        result['options_applied'] = options
        
        return result

# Ejemplo de uso del módulo de lematización y stemming
print("=== MÓDULO 6: LEMATIZACIÓN Y STEMMING ===")

# Crear instancia del lematizador
lemmatizer = TextLemmatizer()

# Texto de ejemplo con términos ambientales variados
sample_text = """
Las contaminaciones ambientales están afectando los ecosistemas naturales.
Las energías renovables ofrecen soluciones sostenibles para reducir las emisiones.
Los científicos estudian la biodiversidad en diferentes regiones protegidas.
Las deforestaciones masivas causan cambios climáticos irreversibles.
Necesitamos implementar tecnologías innovadoras para la conservación.
"""

print("Texto original:")
print(sample_text)
print("\n" + "="*60)

# Lematización con spaCy
print("1. LEMATIZACIÓN CON SPACY:")
spacy_result = lemmatizer.lemmatize_with_spacy(sample_text)
print(f"Texto lematizado: {spacy_result['lemmatized_text']}")
print(f"Cambios realizados: {spacy_result['changes_made']}")
print("Ejemplos de cambios:")
for detail in spacy_result['lemmatization_details'][:5]:
    print(f"  '{detail['original']}' → '{detail['lemma']}' ({detail['pos']})")

print("\n" + "="*40)
print("2. STEMMING CON SNOWBALL:")
snowball_result = lemmatizer.stem_with_snowball(sample_text)
print(f"Texto stemmed: {snowball_result['stemmed_text']}")
print(f"Cambios realizados: {snowball_result['changes_made']}")
print("Ejemplos de cambios:")
for detail in snowball_result['stemming_details'][:5]:
    print(f"  '{detail['original']}' → '{detail['stem']}'")

print("\n" + "="*40)
print("3. ENFOQUE HÍBRIDO:")
hybrid_result = lemmatizer.hybrid_approach(sample_text)
print(f"Texto procesado: {hybrid_result['processed_text']}")
print(f"Cambios realizados: {hybrid_result['changes_made']}")
print("Ejemplos de cambios:")
for detail in hybrid_result['processing_details'][:5]:
    print(f"  '{detail['original']}' → '{detail['processed']}' ({detail['method']})")

print("\n" + "="*60)
print("4. COMPARACIÓN DE MÉTODOS:")
comparison = lemmatizer.compare_methods(sample_text)
print("Tamaños de vocabulario:")
for method, size in comparison['vocabulary_sizes'].items():
    print(f"  {method.capitalize()}: {size} palabras únicas")

print("\nPorcentajes de reducción:")
for method, percentage in comparison['reduction_percentages'].items():
    print(f"  {method.capitalize()}: {percentage}%")

print(f"\nMétodo más agresivo: {comparison['comparison_summary']['most_aggressive']}")
print(f"Método más conservador: {comparison['comparison_summary']['most_conservative']}")
print(f"Método recomendado: {comparison['comparison_summary']['recommended']}")

print("\n" + "="*40)
print("5. EVALUACIÓN DE TÉRMINOS AMBIENTALES:")
env_evaluation = lemmatizer.evaluate_environmental_terms(sample_text)
print("Términos de prueba:", env_evaluation['test_terms'][:3], "...")
print(f"\nSpaCy output: {env_evaluation['spacy_output'][:100]}...")
print(f"Snowball output: {env_evaluation['snowball_output'][:100]}...")
print(f"Hybrid output: {env_evaluation['hybrid_output'][:100]}...")

print("\nPreservación de significado:")
for method, score in env_evaluation['evaluation'].items():
    print(f"  {method}: {score}%")

print("\n" + "="*60)
print("6. PROCESAMIENTO RECOMENDADO (spaCy):")
final_result = lemmatizer.comprehensive_processing(sample_text, method='spacy')
print(f"Texto final procesado:")
print(f"'{final_result['final_processed_text']}'")
print(f"\nEstadísticas finales:")
print(f"- Tokens originales: {final_result['original_tokens']}")
print(f"- Tokens procesados: {final_result['lemmatized_tokens']}")
print(f"- Cambios realizados: {final_result['changes_made']}")


=== MÓDULO 6: LEMATIZACIÓN Y STEMMING ===
Texto original:

Las contaminaciones ambientales están afectando los ecosistemas naturales.
Las energías renovables ofrecen soluciones sostenibles para reducir las emisiones.
Los científicos estudian la biodiversidad en diferentes regiones protegidas.
Las deforestaciones masivas causan cambios climáticos irreversibles.
Necesitamos implementar tecnologías innovadoras para la conservación.


1. LEMATIZACIÓN CON SPACY:
Texto lematizado: el contaminación ambiental estar afectar el ecosistema natural el energía renovable ofrecer solución sostenible para reducir el emisión el científico estudiar el biodiversidad en diferente región protegido el deforestación masivo causar cambio climático irreversible necesitar implementar tecnología innovadora para el conservación
Cambios realizados: 34
Ejemplos de cambios:
  'Las' → 'el' (DET)
  'contaminaciones' → 'contaminación' (NOUN)
  'ambientales' → 'ambiental' (ADJ)
  'están' → 'estar' (AUX)
  'afectando' → 

Modulo 7: Procesamiento con BERT

In [47]:
# ===== MÓDULO 7: PROCESAMIENTO CON BERT =====

class BERTProcessor:
    """
    Módulo para procesamiento y mejora de texto usando BERT.
    Incluye generación de embeddings, mejora de texto, y análisis semántico.
    """
    
    def __init__(self, model_name: str = 'dccuchile/bert-base-spanish-wwm-uncased'):
        self.model_name = model_name
        self.device = device  # Configurado anteriormente
        
        # Inicializar modelos (se cargarán cuando sea necesario)
        self.tokenizer = None
        self.model = None
        self.text_generator = None
        self.summarizer = None
        self.classifier = None
        
        # Configuraciones para diferentes tareas
        self.max_length = 512
        self.environmental_keywords = [
            'medio ambiente', 'sostenible', 'ecológico', 'verde', 'limpio',
            'renovable', 'conservación', 'biodiversidad', 'clima', 'carbono',
            'emisiones', 'contaminación', 'reciclaje', 'energía', 'natural'
        ]
        
        # Templates para mejora de texto
        self.improvement_templates = {
            'titulo': "Mejora este título para que sea más atractivo y claro: {text}",
            'descripcion': "Reescribe esta descripción para que sea más informativa y engaging: {text}",
            'contenido': "Mejora este contenido haciéndolo más profesional y completo: {text}",
            'llamada_accion': "Convierte este texto en una llamada a la acción convincente: {text}",
            'resumen': "Crea un resumen conciso y impactante de: {text}"
        }
    
    def _load_bert_model(self):
        """Carga el modelo BERT si no está cargado"""
        if self.tokenizer is None or self.model is None:
            print(f"Cargando modelo BERT: {self.model_name}")
            try:
                self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
                self.model = BertModel.from_pretrained(self.model_name)
                self.model.to(self.device)
                self.model.eval()
                print("✓ Modelo BERT cargado exitosamente")
            except Exception as e:
                print(f"Error cargando BERT: {e}")
                # Fallback a modelo en inglés
                print("Usando modelo BERT en inglés como fallback...")
                self.model_name = 'bert-base-uncased'
                self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
                self.model = BertModel.from_pretrained(self.model_name)
                self.model.to(self.device)
                self.model.eval()
    
    def _load_text_generator(self):
        """Carga el generador de texto si no está cargado"""
        if self.text_generator is None:
            print("Cargando generador de texto...")
            try:
                # Intentar con modelo en español
                self.text_generator = pipeline(
                    'text-generation',
                    model='DeepESP/gpt2-spanish',
                    tokenizer='DeepESP/gpt2-spanish',
                    device=0 if self.device.type == 'cuda' else -1
                )
                print("✓ Generador de texto en español cargado")
            except Exception as e:
                print(f"Error con modelo español: {e}")
                try:
                    # Fallback a modelo multilingüe
                    self.text_generator = pipeline(
                        'text-generation',
                        model='gpt2',
                        device=0 if self.device.type == 'cuda' else -1
                    )
                    print("✓ Generador de texto (fallback) cargado")
                except Exception as e2:
                    print(f"Error cargando generador: {e2}")
                    self.text_generator = None
    
    def _load_summarizer(self):
        """Carga el resumidor si no está cargado"""
        if self.summarizer is None:
            print("Cargando resumidor...")
            try:
                self.summarizer = pipeline(
                    'summarization',
                    model='facebook/bart-large-cnn',
                    device=0 if self.device.type == 'cuda' else -1
                )
                print("✓ Resumidor cargado")
            except Exception as e:
                print(f"Error cargando resumidor: {e}")
                self.summarizer = None
    
    def generate_embeddings(self, text: str) -> Dict:
        """
        Genera embeddings BERT para el texto
        
        Returns:
            Diccionario con embeddings y estadísticas
        """
        self._load_bert_model()
        
        # Tokenizar texto
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding=True
        )
        
        # Mover a dispositivo
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Generar embeddings
        with torch.no_grad():
            outputs = self.model(**inputs)
            
            # Obtener diferentes tipos de embeddings
            last_hidden_states = outputs.last_hidden_state
            pooler_output = outputs.pooler_output
            
            # Embedding promedio de todos los tokens
            mean_embedding = torch.mean(last_hidden_states, dim=1)
            
            # Embedding del token [CLS]
            cls_embedding = last_hidden_states[:, 0, :]
        
        return {
            'text': text,
            'token_count': inputs['input_ids'].shape[1],
            'embeddings': {
                'cls_embedding': cls_embedding.cpu().numpy(),
                'mean_embedding': mean_embedding.cpu().numpy(),
                'pooler_output': pooler_output.cpu().numpy()
            },
            'embedding_dimension': last_hidden_states.shape[-1],
            'model_used': self.model_name
        }
    
    def calculate_semantic_similarity(self, text1: str, text2: str) -> Dict:
        """
        Calcula similitud semántica entre dos textos usando BERT
        """
        # Generar embeddings para ambos textos
        emb1 = self.generate_embeddings(text1)
        emb2 = self.generate_embeddings(text2)
        
        # Calcular similitud coseno para diferentes tipos de embeddings
        similarities = {}
        
        for emb_type in ['cls_embedding', 'mean_embedding', 'pooler_output']:
            vec1 = emb1['embeddings'][emb_type].flatten()
            vec2 = emb2['embeddings'][emb_type].flatten()
            
            # Similitud coseno
            dot_product = np.dot(vec1, vec2)
            norm1 = np.linalg.norm(vec1)
            norm2 = np.linalg.norm(vec2)
            
            similarity = dot_product / (norm1 * norm2) if norm1 > 0 and norm2 > 0 else 0
            similarities[emb_type] = float(similarity)
        
        return {
            'text1': text1,
            'text2': text2,
            'similarities': similarities,
            'average_similarity': np.mean(list(similarities.values())),
            'most_similar_embedding': max(similarities, key=similarities.get)
        }
    
    def improve_text_with_context(self, text: str, improvement_type: str = 'contenido', 
                                 environmental_focus: bool = True) -> Dict:
        """
        Mejora el texto usando contexto ambiental y BERT
        
        Args:
            text: Texto a mejorar
            improvement_type: Tipo de mejora ('titulo', 'descripcion', 'contenido', etc.)
            environmental_focus: Si enfocar en términos ambientales
        """
        # Analizar el texto original
        original_embeddings = self.generate_embeddings(text)
        
        # Identificar términos ambientales presentes
        environmental_terms_found = [
            term for term in self.environmental_keywords 
            if term.lower() in text.lower()
        ]
        
        # Generar versiones mejoradas
        improvements = []
        
        # Mejora 1: Expansión con términos ambientales
        if environmental_focus:
            expanded_text = self._expand_with_environmental_terms(text, environmental_terms_found)
            improvements.append({
                'version': 'environmental_expansion',
                'text': expanded_text,
                'description': 'Expandido con términos ambientales relevantes'
            })
        
        # Mejora 2: Reestructuración para claridad
        restructured_text = self._restructure_for_clarity(text, improvement_type)
        improvements.append({
            'version': 'restructured',
            'text': restructured_text,
            'description': 'Reestructurado para mayor claridad'
        })
        
        # Mejora 3: Optimización de longitud
        optimized_text = self._optimize_length(text, improvement_type)
        improvements.append({
            'version': 'length_optimized',
            'text': optimized_text,
            'description': 'Optimizado para longitud apropiada'
        })
        
        # Evaluar cada mejora usando BERT
        evaluated_improvements = []
        for improvement in improvements:
            improved_embeddings = self.generate_embeddings(improvement['text'])
            similarity = self.calculate_semantic_similarity(text, improvement['text'])
            
            evaluated_improvements.append({
                **improvement,
                'semantic_similarity': similarity['average_similarity'],
                'token_count': improved_embeddings['token_count'],
                'environmental_terms': len([
                    term for term in self.environmental_keywords 
                    if term.lower() in improvement['text'].lower()
                ])
            })
        
        # Seleccionar la mejor mejora
        best_improvement = max(
            evaluated_improvements, 
            key=lambda x: x['semantic_similarity'] + (x['environmental_terms'] * 0.1)
        )
        
        return {
            'original_text': text,
            'improvement_type': improvement_type,
            'environmental_focus': environmental_focus,
            'original_environmental_terms': environmental_terms_found,
            'all_improvements': evaluated_improvements,
            'best_improvement': best_improvement,
            'improvement_score': best_improvement['semantic_similarity']
        }
    
    def _expand_with_environmental_terms(self, text: str, existing_terms: List[str]) -> str:
        """Expande el texto con términos ambientales relevantes"""
        # Mapeo de términos relacionados
        term_expansions = {
            'medio ambiente': ['ecosistema', 'naturaleza', 'biodiversidad'],
            'sostenible': ['ecológico', 'verde', 'responsable'],
            'contaminación': ['emisiones', 'residuos', 'tóxicos'],
            'energía': ['renovable', 'limpia', 'eficiente'],
            'clima': ['calentamiento global', 'cambio climático'],
            'conservación': ['protección', 'preservación']
        }
        
        expanded_text = text
        
        # Agregar términos relacionados si no están presentes
        for existing_term in existing_terms:
            if existing_term in term_expansions:
                for related_term in term_expansions[existing_term]:
                    if related_term.lower() not in text.lower():
                        # Insertar término relacionado de manera natural
                        expanded_text += f" La {related_term} es fundamental."
                        break  # Solo agregar uno por término existente
        
        return expanded_text
    
    def _restructure_for_clarity(self, text: str, improvement_type: str) -> str:
        """Reestructura el texto para mayor claridad"""
        sentences = text.split('.')
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if improvement_type == 'titulo':
            # Para títulos, hacer más conciso y atractivo
            main_concept = sentences[0] if sentences else text
            return f"{main_concept.strip()}: Solución Ambiental Innovadora"
        
        elif improvement_type == 'descripcion':
            # Para descripciones, estructura problema-solución
            if len(sentences) >= 2:
                return f"{sentences[0]}. Esta situación requiere acción inmediata. {' '.join(sentences[1:])}."
            else:
                return f"{text} Esta iniciativa contribuye significativamente a la sostenibilidad ambiental."
        
        elif improvement_type == 'contenido':
            # Para contenido, agregar estructura y transiciones
            if len(sentences) >= 2:
                restructured = f"En primer lugar, {sentences[0].lower()}. "
                if len(sentences) > 2:
                    restructured += f"Además, {sentences[1].lower()}. "
                    restructured += f"Por último, {' '.join(sentences[2:]).lower()}."
                else:
                    restructured += f"En consecuencia, {sentences[1].lower()}."
                return restructured
            else:
                return f"Es importante destacar que {text.lower()} Esto representa un paso crucial hacia la sostenibilidad."
        
        return text
    
    def _optimize_length(self, text: str, improvement_type: str) -> str:
        """Optimiza la longitud del texto según el tipo"""
        words = text.split()
        
        target_lengths = {
            'titulo': (5, 12),
            'descripcion': (20, 50),
            'contenido': (50, 200),
            'llamada_accion': (10, 25),
            'resumen': (15, 40)
        }
        
        min_len, max_len = target_lengths.get(improvement_type, (20, 100))
        
        if len(words) < min_len:
            # Expandir texto corto
            if improvement_type == 'titulo':
                return f"{text}: Innovación para el Futuro Sostenible"
            else:
                return f"{text} Esta iniciativa representa un avance significativo en la protección ambiental y el desarrollo sostenible."
        
        elif len(words) > max_len:
            # Acortar texto largo
            if improvement_type == 'titulo':
                # Tomar las primeras palabras clave
                key_words = words[:8]
                return ' '.join(key_words)
            else:
                # Resumir manteniendo ideas principales
                sentences = text.split('.')
                main_sentences = sentences[:2] if len(sentences) > 2 else sentences
                return '. '.join(main_sentences).strip() + '.'
        
        return text
    
    def generate_variations(self, text: str, num_variations: int = 3) -> Dict:
        """
        Genera múltiples variaciones del texto
        """
        variations = []
        
        # Variación 1: Enfoque técnico
        technical_variation = self._create_technical_variation(text)
        variations.append({
            'type': 'technical',
            'text': technical_variation,
            'description': 'Enfoque técnico y profesional'
        })
        
        # Variación 2: Enfoque emocional
        emotional_variation = self._create_emotional_variation(text)
        variations.append({
            'type': 'emotional',
            'text': emotional_variation,
            'description': 'Enfoque emocional y persuasivo'
        })
        
        # Variación 3: Enfoque educativo
        educational_variation = self._create_educational_variation(text)
        variations.append({
            'type': 'educational',
            'text': educational_variation,
            'description': 'Enfoque educativo e informativo'
        })
        
        # Evaluar cada variación
        evaluated_variations = []
        for variation in variations:
            similarity = self.calculate_semantic_similarity(text, variation['text'])
            embeddings = self.generate_embeddings(variation['text'])
            
            evaluated_variations.append({
                **variation,
                'semantic_similarity': similarity['average_similarity'],
                'token_count': embeddings['token_count'],
                'environmental_score': self._calculate_environmental_score(variation['text'])
            })
        
        return {
            'original_text': text,
            'variations': evaluated_variations,
            'best_variation': max(evaluated_variations, key=lambda x: x['environmental_score'])
        }
    
    def _create_technical_variation(self, text: str) -> str:
        """Crea una variación con enfoque técnico"""
        technical_terms = {
            'problema': 'desafío técnico',
            'solución': 'implementación estratégica',
            'importante': 'crítico',
            'bueno': 'eficiente',
            'malo': 'ineficiente',
            'ayuda': 'optimiza',
            'hace': 'ejecuta'
        }
        
        technical_text = text
        for original, technical in technical_terms.items():
            technical_text = technical_text.replace(original, technical)
        
        return f"Desde una perspectiva técnica, {technical_text.lower()} Este enfoque garantiza resultados medibles y sostenibles."
    
    def _create_emotional_variation(self, text: str) -> str:
        """Crea una variación con enfoque emocional"""
        emotional_starters = [
            "Imagina un futuro donde",
            "Es hora de actuar:",
            "Nuestro planeta necesita que",
            "Juntos podemos lograr que"
        ]
        
        starter = np.random.choice(emotional_starters)
        return f"{starter} {text.lower()} ¡El momento de actuar es ahora!"
    
    def _create_educational_variation(self, text: str) -> str:
        """Crea una variación con enfoque educativo"""
        return f"Es importante entender que {text.lower()} Los estudios demuestran que estas acciones tienen un impacto positivo significativo en el medio ambiente."
    
    def _calculate_environmental_score(self, text: str) -> float:
        """Calcula un score de relevancia ambiental"""
        score = 0
        text_lower = text.lower()
        
        for keyword in self.environmental_keywords:
            if keyword in text_lower:
                score += 1
        
        # Normalizar por longitud del texto
        words = len(text.split())
        normalized_score = (score / words) * 100 if words > 0 else 0
        
        return min(normalized_score, 10.0)  # Máximo 10
    
    def comprehensive_text_improvement(self, text: str, target_type: str = 'contenido', 
                                     options: Dict = None) -> Dict:
        """
        Mejora completa del texto usando todas las capacidades de BERT
        """
        if options is None:
            options = {
                'environmental_focus': True,
                'generate_variations': True,
                'optimize_length': True,
                'include_embeddings': False,
                'similarity_threshold': 0.7
            }
        
        # Análisis inicial
        original_embeddings = self.generate_embeddings(text) if options.get('include_embeddings') else None
        
        # Mejora principal
        main_improvement = self.improve_text_with_context(
            text, 
            target_type, 
            options.get('environmental_focus', True)
        )
        
        # Generar variaciones si se solicita
        variations = None
        if options.get('generate_variations', True):
            variations = self.generate_variations(main_improvement['best_improvement']['text'])
        
        # Resultado final
        final_text = variations['best_variation']['text'] if variations else main_improvement['best_improvement']['text']
        
        return {
            'original_text': text,
            'target_type': target_type,
            'options': options,
            'main_improvement': main_improvement,
            'variations': variations,
            'final_improved_text': final_text,
            'improvement_summary': {
                'original_length': len(text.split()),
                'final_length': len(final_text.split()),
                'environmental_terms_added': self._calculate_environmental_score(final_text) - self._calculate_environmental_score(text),
                'semantic_preservation': main_improvement['improvement_score']
            },
            'original_embeddings': original_embeddings
        }

# Ejemplo de uso del módulo BERT
print("=== MÓDULO 7: PROCESAMIENTO CON BERT ===")

# Crear instancia del procesador BERT
bert_processor = BERTProcessor()

# Texto de ejemplo para mejorar
sample_text = "El reciclaje es importante para el planeta. Debemos cuidar el medio ambiente."

print("Texto original:")
print(f"'{sample_text}'")
print("\n" + "="*60)

# Generar embeddings
print("1. GENERACIÓN DE EMBEDDINGS:")
embeddings_result = bert_processor.generate_embeddings(sample_text)
print(f"Tokens procesados: {embeddings_result['token_count']}")
print(f"Dimensión de embeddings: {embeddings_result['embedding_dimension']}")
print(f"Modelo usado: {embeddings_result['model_used']}")

# Mejora de texto con contexto
print("\n2. MEJORA DE TEXTO CON CONTEXTO:")
improvement_result = bert_processor.improve_text_with_context(
    sample_text, 
    improvement_type='contenido',
    environmental_focus=True
)

print(f"Mejor mejora encontrada:")
print(f"Versión: {improvement_result['best_improvement']['version']}")
print(f"Texto mejorado: '{improvement_result['best_improvement']['text']}'")
print(f"Score de similitud: {improvement_result['improvement_score']:.3f}")
print(f"Términos ambientales: {improvement_result['best_improvement']['environmental_terms']}")

# Generar variaciones
print("\n3. GENERACIÓN DE VARIACIONES:")
variations_result = bert_processor.generate_variations(sample_text)
print("Variaciones generadas:")
for var in variations_result['variations']:
    print(f"\n  {var['type'].upper()}:")
    print(f"  Texto: '{var['text'][:100]}{'...' if len(var['text']) > 100 else ''}'")
    print(f"  Score ambiental: {var['environmental_score']:.2f}")
    print(f"  Similitud semántica: {var['semantic_similarity']:.3f}")

# Mejora completa
print("\n" + "="*60)
print("4. MEJORA COMPLETA DEL TEXTO:")

final_result = bert_processor.comprehensive_text_improvement(
    sample_text,
    target_type='descripcion',
    options={
        'environmental_focus': True,
        'generate_variations': True,
        'optimize_length': True,
        'include_embeddings': False
    }
)

print("RESULTADO FINAL:")
print(f"Texto original: '{final_result['original_text']}'")
print(f"Texto mejorado: '{final_result['final_improved_text']}'")

print(f"\nResumen de mejoras:")
summary = final_result['improvement_summary']
print(f"- Longitud original: {summary['original_length']} palabras")
print(f"- Longitud final: {summary['final_length']} palabras")
print(f"- Términos ambientales añadidos: {summary['environmental_terms_added']:.1f}")
print(f"- Preservación semántica: {summary['semantic_preservation']:.3f}")

# Ejemplo con diferentes tipos de texto
print("\n" + "="*40)
print("5. EJEMPLOS CON DIFERENTES TIPOS:")

test_cases = [
    ("Energía solar", "titulo"),
    ("Las plantas purifican el aire", "descripcion"),
    ("plantas purificar aire","descripcion"),
    ("Recicla para salvar el planeta", "llamada_accion")
]

for text, text_type in test_cases:
    result = bert_processor.comprehensive_text_improvement(text, text_type)
    print(f"\n{text_type.upper()}:")
    print(f"Original: '{text}'")
    print(f"Mejorado: '{result['final_improved_text']}'")


=== MÓDULO 7: PROCESAMIENTO CON BERT ===
Texto original:
'El reciclaje es importante para el planeta. Debemos cuidar el medio ambiente.'

1. GENERACIÓN DE EMBEDDINGS:
Cargando modelo BERT: dccuchile/bert-base-spanish-wwm-uncased
✓ Modelo BERT cargado exitosamente
Tokens procesados: 16
Dimensión de embeddings: 768
Modelo usado: dccuchile/bert-base-spanish-wwm-uncased

2. MEJORA DE TEXTO CON CONTEXTO:
Mejor mejora encontrada:
Versión: length_optimized
Texto mejorado: 'El reciclaje es importante para el planeta. Debemos cuidar el medio ambiente. Esta iniciativa representa un avance significativo en la protección ambiental y el desarrollo sostenible.'
Score de similitud: 0.836
Términos ambientales: 3

3. GENERACIÓN DE VARIACIONES:
Variaciones generadas:

  TECHNICAL:
  Texto: 'Desde una perspectiva técnica, el reciclaje es crítico para el planeta. debemos cuidar el medio ambi...'
  Score ambiental: 10.00
  Similitud semántica: 0.774

  EMOTIONAL:
  Texto: 'Imagina un futuro donde el recicl

Modulo 8: Generacion de Salida y Sistema Completo

In [48]:
# ===== SISTEMA COMPLETO MEJORADO =====


class TextMiningSystem:
    def __init__(self):
        self.ingestion = TextIngestion()
        self.cleaner = TextCleaner()
        self.tokenizer = TextTokenizer()
        self.normalizer = TextNormalizer()
        self.noise_remover = NoiseRemover()  # Versión corregida
        self.lemmatizer = TextLemmatizer()
        self.bert_processor = BERTProcessor()
    
    def process_text_complete_enhanced(self, text: str, content_type: str = 'contenido', track_steps: bool = False) -> Dict:
        """Procesamiento mejorado con mejor control de calidad"""
        
        # Configuraciones específicas por tipo de contenido
        processing_configs = {
            'titulo': {
                'aggressive_cleaning': False,
                'preserve_length': True,
                'min_similarity': 0.7
            },
            'descripcion': {
                'aggressive_cleaning': False,
                'preserve_length': False,
                'min_similarity': 0.6
            },
            'contenido': {
                'aggressive_cleaning': True,
                'preserve_length': False,
                'min_similarity': 0.5
            }
        }
        
        config = processing_configs.get(content_type, processing_configs['contenido'])
        
        # Validación inicial
        original_input = text
        text = self._validate_text(text, "entrada inicial")
        if text is None:
            text = original_input
        
        intermediate_results = {}
        processing_steps = []
        
        try:
            # Paso 1: Ingesta (sin cambios)
            print("Ejecutando Paso 1: Ingesta...")
            ingestion_result = self.ingestion.ingest_manual_text(text)
            current_text = ingestion_result['original_text']
            
            if track_steps:
                intermediate_results['ingestion'] = current_text
                processing_steps.append({
                    'step': 'ingesta',
                    'metrics': {'length': ingestion_result['length']}
                })
            
            # Paso 2: Limpieza (menos agresiva)
            print("Ejecutando Paso 2: Limpieza...")
            cleaning_options = {
                'remove_urls': True,
                'remove_emails': True,
                'remove_phones': True,
                'remove_html': True,
                'remove_special_chars': not config['preserve_length'],
                'keep_punctuation': True,
                'normalize_whitespace': True,
                'normalize_punctuation': True,
                'remove_newlines': True,
                'fix_encoding': True
            }
            cleaning_result = self.cleaner.basic_clean(current_text, cleaning_options)
            current_text = cleaning_result['cleaned_text']
            
            if track_steps:
                intermediate_results['cleaning'] = current_text
                processing_steps.append({
                    'step': 'limpieza',
                    'metrics': {'reduction_percentage': cleaning_result['reduction_percentage']}
                })
            
            # Paso 3: Tokenización (sin cambios)
            print("Ejecutando Paso 3: Tokenización...")
            tokens = self.tokenizer.tokenize_words(current_text, 'spacy')
            if not tokens:
                tokens = current_text.split()
            
            if track_steps:
                intermediate_results['tokenization'] = ' '.join(tokens[:20]) + ('...' if len(tokens) > 20 else '')
                processing_steps.append({
                    'step': 'tokenización',
                    'metrics': {'token_count': len(tokens)}
                })
            
            # Paso 4: Normalización (mejorada)
            print("Ejecutando Paso 4: Normalización...")
            normalization_options = {
                'to_lowercase': True,
                'remove_accents': False,  # Preservar acentos para mejor legibilidad
                'expand_contractions': True,
                'expand_abbreviations': True,
                'normalize_numbers': 'keep',
                'normalize_case_patterns': True,
                'normalize_environmental_terms': True
            }
            normalization_result = self.normalizer.normalize_tokens(tokens, normalization_options)
            normalized_tokens = normalization_result.get('normalized_tokens', tokens)
            current_text = ' '.join(normalized_tokens)
            
            if track_steps:
                intermediate_results['normalization'] = current_text
                processing_steps.append({
                    'step': 'normalización',
                    'metrics': {'token_count_change': normalization_result.get('token_count_change', 0)}
                })
            
            # Paso 5: Eliminación de ruido (configuración adaptativa)
            print("Ejecutando Paso 5: Eliminación de ruido...")
            noise_options = {
                'remove_stopwords': config['aggressive_cleaning'],
                'aggressive_stopwords': False,
                'remove_short_words': True,
                'min_word_length': 2,
                'remove_noise_words': True,
                'remove_by_frequency': False,
                'remove_non_alphabetic': False,
                'remove_by_pos': False,
                'remove_environmental_noise': False,
                'use_advanced_removal': False
            }
            noise_removal_result = self.noise_remover.comprehensive_noise_removal(current_text, noise_options)
            current_text = noise_removal_result['cleaned_text']
            
            if track_steps:
                intermediate_results['noise_removal'] = current_text
                processing_steps.append({
                    'step': 'eliminación_ruido',
                    'metrics': {'reduction_percentage': noise_removal_result['reduction_percentage']}
                })
            
            # Paso 6: Lematización (mejorada)
            print("Ejecutando Paso 6: Lematización...")
            lemmatization_options = {
                'preserve_environmental_terms': True,
                'preserve_verbs': True,
                'min_word_length': 2,
                'remove_duplicates': False,
                'maintain_grammar': True
            }
            lemmatization_result = self.lemmatizer.comprehensive_processing_improved(
                current_text, 
                method='spacy_improved',
                options=lemmatization_options
            )
            current_text = lemmatization_result['final_processed_text']
            
            if track_steps:
                intermediate_results['lemmatization'] = current_text
                processing_steps.append({
                    'step': 'lematización',
                    'metrics': {'changes_made': lemmatization_result.get('changes_made', 0)}
                })
            
            # Paso 7: Procesamiento con BERT (mejorado)
            print("Ejecutando Paso 7: Procesamiento con BERT...")
            bert_options = {
                'environmental_focus': True,
                'generate_variations': False,
                'optimize_length': True,
                'preserve_meaning': True,
                'min_similarity_threshold': config['min_similarity']
            }
            bert_result = self.bert_processor.comprehensive_text_improvement_enhanced(
                current_text,
                target_type=content_type,
                options=bert_options
            )
            final_text = bert_result['final_improved_text']
            
            if track_steps:
                intermediate_results['bert_processing'] = final_text
                processing_steps.append({
                    'step': 'bert_processing',
                    'metrics': {
                        'semantic_preservation': bert_result['improvement_summary']['semantic_preservation'],
                        'coherence_score': bert_result['improvement_summary']['coherence_score'],
                        'grammar_score': bert_result['improvement_summary']['grammar_score']
                    }
                })
            
            # Evaluación de calidad mejorada
            print("Ejecutando evaluación de calidad...")
            evaluation = self._evaluate_quality_enhanced(text, final_text, bert_result['improvement_summary'])
            
            # Generar recomendaciones
            recommendations = self._generate_recommendations_enhanced(evaluation, processing_steps)
            
            return {
                'original_text': text,
                'final_text': final_text,
                'content_type': content_type,
                'intermediate_results': intermediate_results if track_steps else {},
                'processing_steps': processing_steps if track_steps else [],
                'evaluation': evaluation,
                'recommendations': recommendations,
                'bert_details': bert_result,
                'processing_config': config
            }
            
        except Exception as e:
            print(f"❌ Error durante el procesamiento: {str(e)}")
            return self._create_fallback_result(text, content_type, str(e))
    
    def _evaluate_quality_enhanced(self, original_text: str, final_text: str, improvement_summary: Dict) -> Dict:
        """Evaluación de calidad mejorada"""
        try:
            original_words = len(original_text.split()) if original_text else 0
            final_words = len(final_text.split()) if final_text else 0
            
            # Usar métricas del improvement_summary si están disponibles
            semantic_similarity = improvement_summary.get('semantic_preservation', 0.7)
            coherence_score = improvement_summary.get('coherence_score', 0.7)
            grammar_score = improvement_summary.get('grammar_score', 0.7)
            
            # Score ambiental
            env_score = self.bert_processor._calculate_environmental_score(final_text)
            
            # Evaluación de legibilidad mejorada
            readability_score = self._calculate_readability(final_text)
            
            # Score general ponderado
            overall_quality = (
                semantic_similarity * 3 +
                coherence_score * 2.5 +
                grammar_score * 2 +
                (env_score / 10) * 1.5 +
                (readability_score / 10) * 1
            )
            
            return {
                'semantic_similarity': semantic_similarity * 10,
                'coherence': coherence_score * 10,
                'grammar': grammar_score * 10,
                'environmental_relevance': env_score,
                'readability': readability_score,
                'length_optimization': min(10, max(1, 10 - abs(final_words - 15) / 5)) if final_words > 0 else 5.0,
                'overall_quality': min(10, overall_quality)
            }
            
        except Exception as e:
            print(f"❌ Error en evaluación: {e}")
            return {
                'semantic_similarity': 7.0,
                'coherence': 7.0,
                'grammar': 7.0,
                'environmental_relevance': 5.0,
                'readability': 7.0,
                'length_optimization': 7.0,
                'overall_quality': 6.8
            }
    
    def _calculate_readability(self, text: str) -> float:
        """Calcula un score de legibilidad básico"""
        if not text:
            return 5.0
        
        words = text.split()
        sentences = text.split('.')
        
        # Métricas básicas
        avg_words_per_sentence = len(words) / len(sentences) if sentences else len(words)
        avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
        
        # Score basado en complejidad
        readability = 10
        
        # Penalizar oraciones muy largas
        if avg_words_per_sentence > 20:
            readability -= 2
        elif avg_words_per_sentence > 15:
            readability -= 1
        
        # Penalizar palabras muy largas
        if avg_word_length > 8:
            readability -= 2
        elif avg_word_length > 6:
            readability -= 1
        
        # Bonificar longitud apropiada
        if 5 <= len(words) <= 20:
            readability += 1
        
        return max(1.0, min(10.0, readability))
    
    def _generate_recommendations_enhanced(self, evaluation: Dict, processing_steps: List) -> List[str]:
        """Genera recomendaciones mejoradas"""
        recommendations = []
        
        try:
            if evaluation.get('semantic_similarity', 0) < 7:
                recommendations.append("Ajustar parámetros de lematización para preservar mejor el significado original")
            
            if evaluation.get('coherence', 0) < 7:
                recommendations.append("Mejorar la coherencia textual manteniendo la estructura gramatical")
            
            if evaluation.get('grammar', 0) < 7:
                recommendations.append("Revisar la gramática del texto procesado")
            
            if evaluation.get('environmental_relevance', 0) < 5:
                recommendations.append("Incorporar más términos ambientales relevantes")
            
            if evaluation.get('readability', 0) < 6:
                recommendations.append("Simplificar el vocabulario para mejorar la legibilidad")
            
            if evaluation.get('length_optimization', 0) < 7:
                recommendations.append("Ajustar la longitud según el tipo de contenido")
            
            if not recommendations:
                recommendations.append("El texto ha sido procesado exitosamente con alta calidad")
                
        except Exception as e:
            recommendations.append(f"Error generando recomendaciones: {str(e)}")
        
        return recommendations
